In [20]:
from _collections_abc import Sequence, Iterable
from abc import ABC
from sage.combinat.finite_state_machine import FiniteStateMachine, FSMState, FSMTransition, Automaton
from sage.combinat.subset import powerset
import matplotlib
import networkx as nx
import matplotlib.pyplot as plt
from itertools import product
import copy

In [21]:
class WordGenerator:
    def __init__(self, commutation_dict: dict[str: list], order_dict: dict[str: int]):
        self.c_map = commutation_dict
        self.o_map = order_dict

    def word(self, word):
        return Word(word, self.c_map, self.o_map)

    def horocyclic_word(self, subwordlist, mode):
        return HorocyclicWord(subwordlist, mode, self.c_map, self.o_map)


class Word(Sequence):
    def __init__(self, word, commutation_dict: dict[str: list], order_dict: dict[str: int]):
        if word is list:
            self.word_as_list = word
        elif isinstance(word, Iterable):
            self.word_as_list = list(word)
        else:
            raise TypeError("argument 'word' is not iterable")
        self.c_map = commutation_dict
        self.o_map = order_dict
        self.alphabet = set().union(letter for letter in order_dict)
        # super.__init__()

    def __getitem__(self, item):
        return self.word_as_list[item]

    def __len__(self):
        return len(self.word_as_list)

    def __str__(self):
        return ''.join(str(a) for a in self.word_as_list)

    def __eq__(self, other):
        if isinstance(other, Word):
            return self.word_as_list == other.word_as_list
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    def __repr__(self):
        return('Word(' + str(self.word_as_list) +')')

    def word_as_tuple (self) -> tuple:
        return (tuple(self.word_as_list))
    
    def copy(self):
        copy_word_list = list(letter for letter in self.word_as_list)
        return Word(copy_word_list, self.c_map, self.o_map)

    def append(self, value: str) -> None:
        self.word_as_list.append(value)

    def shortlex_append(self, value: str):
        optimal_insert_idx = len(self.word_as_list)
        for i in reversed(range(len(self.word_as_list))):
            if value not in self.c_map[self.word_as_list[i]]:
                break
            if self.o_map[self.word_as_list[i]] > self.o_map[value]:
                optimal_insert_idx = i
        self.word_as_list.insert(optimal_insert_idx, value)
        return self

    def insert(self, index: int, value: str):
        self.word_as_list.insert(index, value)
        return self

    def __neighborhood(self, letter: str) -> set:
        return set(self.c_map[letter])

class HorocyclicWord(Sequence):
    def __init__(self, subword_list:list, mode:bool, commutation_dict: dict[str: list], order_dict: dict[str: int]):
        '''
        :param subwordlist: a list of 4 lists. Each inner list should be a list of strings.
        :param mode: True if the word is of form 1234, False if the word is of form 1256. Note that the class will not check that the word is of the desired form.
        '''
        if len(subword_list) != 4:
            raise ValueError ('Please specify exactly 4 (possibly empty) subwords')

        self.c_map = commutation_dict
        self.o_map = order_dict
        self.alphabet = set().union(letter for letter in order_dict)
        self.mode = mode
        
        self.SubwordList = subword_list 
        self.final_subword = 0
        for i in reversed(range (0, 4)):
            if subword_list[i] != []:
                self.final_subword = i
                break

        self.word_as_list = subword_list[0]+subword_list[1]+subword_list[2]+subword_list[3]
                
    def __getitem__(self, item):
        return self.SubwordList[item]

    def __len__(self):
        return len(self.word_as_list)

    def __str__(self):
        return ''.join(str(a) for a in self.word_as_list)

    def __eq__(self, other):
        if isinstance(other, HorocyclicWord):
            return self.SubwordList == other.SubwordList
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    def __repr__(self):
        return('HorocyclicWord(' + self.SubwordList +')')
    
    def copy(self):
        copy_subword_list = [list(letter for letter in self.SubwordList[0]), list(letter for letter in self.SubwordList[1]), list(letter for letter in self.SubwordList[2]), list(letter for letter in self.SubwordList[3])]
        return HorocyclicWord(copy_subword_list, self.mode, self.c_map, self.o_map)
    
    def append(self, letter: str, position: int) -> None:
        if self.final_subword > position:
            raise ValueError ('cannot append to subwords which have ended')
        self.SubwordList[position].append(letter)
        self.word_as_list.append(letter)
        self.final_subword = position

    

In [22]:
#Not used in the below. Keeping around only in case it's needed.

def multi_epsilon_successors(concatenated_machine: Automaton, start_state: FSMState) -> list[FSMTransition]:
        '''
        Given an input automaton and a state in that automaton, this function gives a list of composite transitions that arise as a (possibly empty) sequence of epsilon transitions followed by one non-epsilon transition.

        :param concatenated_machine: an automaton with epsilon states. If there are epsilon loops this might not work, but should at least terminate.
        :param start_state: a state in concatenated_machine whose successors we would like.

        :return: a list of FSMTransitions from start_state to end_state, where end_state can be reached from start_state by a sequence of epsilon transitions followed by a single non-epsilon transition.
        The list entries are labeled by the labels of their non-epsilon transitions.
        '''

        MultiEpsilonTransitionList = []
        
        if not start_state in concatenated_machine.states():
            raise ValueError ("The starting state does not appear in the provided automaton")

        for epsilon_successor_state in set(concatenated_machine.epsilon_successors(start_state).keys()).union({start_state}):
            for non_epsilon_transition in concatenated_machine.transitions(concatenated_machine.state(epsilon_successor_state)):
                if non_epsilon_transition.word_in == []:
                    continue
                MultiEpsilonTransitionList.append(FSMTransition(start_state, non_epsilon_transition.to_state, non_epsilon_transition.word_in))
        
        #print('Total non-epsilon trasitions are')
        #print(MultiEpsilonTransitionList)
        return MultiEpsilonTransitionList
    
def de_epsilonize(concatenated_machine: Automaton) ->Automaton:
        '''
        Unfortunately, sage.combinat.finite_sate_machine.FiniteStateMachine.concatenation relies on the creation of epsilon-transitions, which means that intersections of concatenated languages are not well-defined.
        Therefore, to make the language of horocyclic suffices, we will use this function that deletes epsilon transitions.
        This uses the fact that the beginning of each word w_i is unambiguous, since w_{i+1} always begins with a letter that was not alowed to be in w_i. Do not use this function if this is not the case. 
        (An alternative implementation involves constructing a nondeterministic concatenation machine and then determinizing it. This is more work).
        
        :param concatenated_machine: an Automaton with some number of epsilon transitions.
        
        :return: an Automaton which accepts the same language as concatenated_machine, without epsilon transitions.
        This may not be deterministic. If it is a concatenation of other machines with languages L_1, ...L_k such that words in L_i never end with letters that words in L_{i+1} begin with, then it will be deterministic.
        
        '''
        TransitionList = []
        FinishedStates = []
        Frontier = []
        
        for StartingState in concatenated_machine.initial_states():
            NewTransitions = multi_epsilon_successors(concatenated_machine, StartingState)
            for new_transition in NewTransitions:
                Frontier.append(new_transition.to_state)
            TransitionList.extend(NewTransitions)
            FinishedStates.append(StartingState)

        while len(Frontier)>0:
            SourceState = Frontier.pop(0)

            # We have already considered source and all its outgoing edges, we can skip it
            if SourceState in FinishedStates:
                continue
            NewTransitions = multi_epsilon_successors(concatenated_machine, SourceState)
            for new_transition in NewTransitions:
                Frontier.append(new_transition.to_state)
            TransitionList.extend(NewTransitions)
            FinishedStates.append(SourceState)

        #print('de-epsilonized TransitionList is')
        #print(TransitionList)
    
        return Automaton(TransitionList)
        
def unambiguous_concatenation(first_machine: Automaton, second_machine: Automaton) -> Automaton:
    '''
    A custom concatenation function for automata M_1, M_2 with languages L_1 and L_2, satisfying the following:
    (1) M_1 and M_2 have no epsilon transitions
    (2) No word of L_1 ends with a letter that a word of L_2 begins with.
    (3) L_2 contains the empty word.
    The code is a modification of the .concatenation(other) method from sage.combinat.finite_state_machine.
    If (2) is not satisfied, this should output a non-deterministic automaton recognizing L_1L_2
    
    :param first_machine: a non-empty Automaton recognizing the language L_1
    :param second_machine: a non-empty Automaton recognizing the language L_2

    :return: an automaton without epsilon transitions recognizing the concatenated language L_1L_2
    '''
        
    result = self.empty_copy()
    first_states = {}
    second_states = {}
    for s in first_machine.iter_states():
        new_state = s.relabeled((0, s.label()))
        first_states[s] = new_state
        result.add_state(new_state)
        #Unlike in sage.combinat.finite_state_machine, we allow these states to be final exactly when the state they are copying is final. This means that each word w_1 in L_1 will again be accepted by the result, without needing an epsilon transition to a starting state of M_2

    
    for s in second_machine.iter_states():
        new_state = s.relabeled((1, s.label()))
        new_state.is_initial = False
        second_states[s] = new_state
        result.add_state(new_state)

    for t in first_machine.iter_transitions():
        result.add_transition(first_states[t.from_state],
                              first_states[t.to_state],
                              t.word_in,
                              t.word_out)

    for t in second_machine.iter_transitions():
        result.add_transition(second_states[t.from_state],
                              second_states[t.to_state],
                              t.word_in,
                              t.word_out)

    for s in first_machine.iter_final_states():
        first_state = first_states[s]
        for t in second_machine.iter_initial_states():
            second_state = second_states[t]
            #Unlike in sage.combinat.finite_state_machine, we create labeled transitions directly to those states in second_machine that immediately follow initial states. This avoids the creation of epsilon transitions.
            for transition in second_machine.transitions(second_state):
                result.add_transition(first_state,
                                      transition.to_state,
                                      transition.word_in,
                                      transition.word_out)
        try:
            result.input_alphabet = list(set(self.input_alphabet)
                                         | set(other.input_alphabet))
        except TypeError:
            # e.g. None or unhashable letters
            result.input_alphabet = None

        return result



In [23]:
class ConcatableAutomaton(Automaton):
    #This is the class for the automata we will use in this paper.

    def __init__(self, *args, **kwargs):
        """
        This is the initialization statement copied for automata
        """
        super().__init__(*args, **kwargs)
    
    def unambiguous_concatenation(self, other: 'ConcatableAutomaton') -> 'ConcatableAutomaton':
        '''
        A custom concatenation function for ConcatableAutomata M_1, M_2 with languages L_1 and L_2, satisfying the following:
        (1) M_1 and M_2 have no epsilon transitions
        (2) No word of L_1 ends with a letter that a word of L_2 begins with.
        (3) L_2 contains the empty word.
        The code is a modification of the .concatenation(other) method from sage.combinat.finite_state_machine.
        If (2) is not satisfied, this should output a non-deterministic automaton recognizing L_1L_2
    
        :param other: a non-empty Concatable Automaton.

        :return: a ConcatableAutomaton without epsilon transitions recognizing the concatenated language L_1L_2
        '''

        TransitionList = []
        first_states = {}
        second_states = {}
        for s in self.iter_states():
            new_state = s.relabeled((0, s.label()))
            first_states[s] = new_state
            #Unlike in sage.combinat.finite_state_machine, we allow these states to be final when the state they are copying is final. This means that each word w_1 in L_1 will again be accepted by the result, without needing an epsilon transition to a starting state of M_2

    
        for s in other.iter_states():
            new_state = s.relabeled((1, s.label()))
            new_state.is_initial = False
            second_states[s] = new_state

        for t in self.iter_transitions():
            NewTransition = FSMTransition(first_states[t.from_state],
                                          first_states[t.to_state],
                                          t.word_in,
                                          t.word_out)
            TransitionList.append(NewTransition)

        for t in other.iter_transitions():
            if (t.from_state).is_initial:
                continue
            NewTransition = FSMTransition(second_states[t.from_state],
                                          second_states[t.to_state],
                                          t.word_in,
                                          t.word_out)
            TransitionList.append(NewTransition)

        for s in self.iter_final_states():
            first_state = first_states[s]
            for t in other.iter_initial_states():
                second_state = second_states[t]
                #Unlike in sage.combinat.finite_state_machine, we create labeled transitions directly to those states in other that immediately follow initial states. This avoids the creation of epsilon transitions.
                for transition in other.transitions(t):
                    NewTransition = FSMTransition (first_state,
                                                   second_states[transition.to_state],
                                                   transition.word_in,
                                                   transition.word_out)
                    TransitionList.append(NewTransition)
   
        return ConcatableAutomaton(TransitionList)

    def concatable_intersection(self, other: 'ConcatableAutomaton', only_accessible_components = True) -> 'ConcatableAutomaton':
        #Compute self.intersection(other) as in sage.combinat.finite_state_machine, but return ConcatableAutomaton

        return ConcatableAutomaton(self.intersection(other, only_accessible_components))
    
    def _interspersal(self, other: 'ConcatableAutomaton') -> 'ConcatableAutomaton':
        '''Let the languages L_0 and L_1 be recognized respectively by self and other. This method takes the union of the even-length words in L_0 and the odd-length words in L_1
        
        :param self: A ConcatableAutomaton.
        :param other: A ConcatableAutomaton.

        :return: A ConcatableAutomaton recognizing the even-length words accepted by self and the odd length words accepted by other.     
        '''
        
        self.input_alphabet = set(self.input_alphabet).union(set(other.input_alphabet))
        other.input_alphabet = set(self.input_alphabet).union(set(other.input_alphabet))
        
        CompleteSelf = self.completion()
        CompleteOther = other.completion()
        
        #Define an automaton to keep track of the parity of the lengths of words.
        
        EvenState = FSMState(0, is_initial = True, is_final = True)
        OddState = FSMState(1, is_initial = False, is_final = True)
        TransitionList = []
        for letter in set(self.input_alphabet).union(set(other.input_alphabet)):
            TransitionList.append(FSMTransition(EvenState, OddState, letter))
            TransitionList.append(FSMTransition(OddState, EvenState, letter))
        ParityMachine = ConcatableAutomaton(TransitionList)
        
        ApproxMachine = ParityMachine.concatable_intersection(CompleteSelf.concatable_intersection(CompleteOther))
        #Because all these machines are complete, self.concatable_intersection(other) will still track its progress in other of a word rejected by self, and vice versa.
        #States (n, (self.state(spam), other.state(spam))) are final iff both (self.state(spam)) and (other.state(spam)) are, where n is either EvenState or OddState. So this automaton has the correct states but no final states.
        
        for state in ApproxMachine.iter_states():
            #Labels are of the form (n, (self.state(spam), other.state(spam))). We need n.label() to get the parity. 
            parity = (state.label()[0]).label()
            state.is_final = state.label()[1].label()[parity].is_final

        return ApproxMachine

"""
    Commented out for now. It appears to be correct, but it was giving an unhashable type error, which suggests that some transitions are not labeled as intended.
    
    def transition_dict(self, state:FSMState) -> dict:
        '''
        Generate a dictionary describing the FSMTransitions of self that start at the provided state.
        Keys to this dictionary are letters of the input alphabet, and values are instances of FSMtransition starting at state and labeled by the key value.
        
        :param state: a state in self.
        :return: A dictionary whose keys are the letters of the input alphabet and whose values are the transitions from state labeled by those keys.
        '''

        if not self.is_deterministic:
            raise ValueError('Not implemented for non-deterministic automata')
            
        TransitionDict = {}
        for transition in self.iter_transitions(state):
            TransitionDict[transition.word_in]=transition
        return TransitionDict

"""


"\n    Commented out for now. It appears to be correct, but it was giving an unhashable type error, which suggests that some transitions are not labeled as intended.\n    \n    def transition_dict(self, state:FSMState) -> dict:\n        '''\n        Generate a dictionary describing the FSMTransitions of self that start at the provided state.\n        Keys to this dictionary are letters of the input alphabet, and values are instances of FSMtransition starting at state and labeled by the key value.\n        \n        :param state: a state in self.\n        :return: A dictionary whose keys are the letters of the input alphabet and whose values are the transitions from state labeled by those keys.\n        '''\n\n        if not self.is_deterministic:\n            raise ValueError('Not implemented for non-deterministic automata')\n            \n        TransitionDict = {}\n        for transition in self.iter_transitions(state):\n            TransitionDict[transition.word_in]=transition\n   

In [24]:
class Rips_FSM_Generator:
    def __init__(self, commutation_dict:dict[str, set], order_dict: dict[str, int], ray: list[str]):
        
        '''
        This object will make the necessary automata for the Rips graph on a horosphere.
    
        :param commutation_dict: A dictionary representation of a defining graph. A letter (key) is associated with a list of letters (value) that the key letter commutes with. This list should not include the key letter.
        Note: this program does not check that commutation_dict is symmetrical. You need to make sure that if your input allows a_i to commute with a_j, then a_j is also allowed to commute with a_i.
        :param order_dict: A dictionary representation of a total ordering on the letter in the defining graph. A letter (key) is associated with an index (value) that represents that letters relative position in the ordering.
        :param ray: A list of two characters that will be used to generate the chosen ray to infinity. In the paper these are referred to as a_i and a_j respectively.
        '''

        if not len(ray) == 2:
            raise ValueError ("The defining ray should have two letters")
        elif ray [1] in commutation_dict[ray[0]]:
            raise ValueError ("The defining ray consists of two letters which commute")
        
        self.c_map = commutation_dict
        self.o_map = order_dict
        self.alphabet = set().union(letter for letter in self.o_map)
        self.ray = ray

        StringOfAllLetters = ''.join(letter for letter in self.alphabet)
        if '-' in StringOfAllLetters:
            raise ValueError ('''The character '-' is reserved to be the blank character''')
        if ',' in StringOfAllLetters:
            raise ValueError('''The character ',' is reserved for delimiting lists''')
        if '_' in StringOfAllLetters:
            raise ValueError(''' The character '_' is reserved for the word breaks in horocyclic suffices''')
        
        #the lesser_star dictionary agrees with the function Star_< in the paper. When passed a key of a letter, it returns the set of letters that commute with and precede the key. Note the strict inequality.
        self.lesser_star = {}
        for letter in self.alphabet:
            LesserStarOfLetter = set(filter(lambda x: self.o_map[x] < self.o_map[letter], self.c_map[letter]))
            self.lesser_star[letter] = LesserStarOfLetter
        self.greater_star = {}
        for letter in self.alphabet:
            self.greater_star[letter] = self.c_map[letter].difference(self.lesser_star[letter])

    
    def __first_letter_excluder(self, excluded_letters:set) -> ConcatableAutomaton:
        '''
        Generate an FSM that prevents a chosen set of letters from reaching the beginning of the word without cancellation. The states in this machine represent which letters could still reach the beginning of the word.
        
        :param excluded_letters: A set of letters which that accepted words should not begin with.
                
        :return: An automaton that accepts words which, without cancellation, cannot be commuted to begin with a letter in excluded_letters
        '''
        if excluded_letters == self.alphabet:
            print("It appears that every letter has been excluded. Returning an automaton that accepts only the empty string")
            SingleState = FSMState('origin', is_initial = True, is_final = True)
            return ConcatableAutomaton({SingleState:[]})
        sorted_excluded_letters = sorted(excluded_letters, key = lambda x: self.o_map[x])
        TransitionList = []
        Frontier = []
        StartStateName = tuple(sorted_excluded_letters)
        # StartStateName = ",".join(str(letter) for letter in sorted(excluded_letters, key = lambda x: self.o_map[x]))
        StartState = FSMState(StartStateName, is_initial = True, is_final = True)
        for nextletter in self.alphabet.difference(excluded_letters):
            NextName = tuple(sorted(excluded_letters.intersection(self.c_map[nextletter]), key = lambda x: self.o_map[x]))
            NextState = FSMState(NextName, is_initial = (NextName == StartStateName), is_final = True)
            TransitionList.append(FSMTransition(StartState, NextState, nextletter))
            Frontier.append(NextState)
            
        #FinishedStates keeps track of which vertices we have already found all the edges for.
        
        FinishedStates = [StartState]
        while len(Frontier)>0:
            SourceState = Frontier.pop(0)
            if SourceState in FinishedStates:
                continue
        
            SourceSet = set(SourceState.label())
            FinishedStates.append(SourceState)
            for nextletter in self.alphabet.difference(SourceSet):
                NextName = tuple(sorted(SourceSet.intersection(self.c_map[nextletter]), key = lambda x: self.o_map[x]))
                NextState = FSMState(NextName, is_initial = (NextName == StartStateName), is_final = True)
                Frontier.append(NextState)
                TransitionList.append(FSMTransition(SourceState, NextState, nextletter))
       
        #print('TransitionList is')
        #print(TransitionList)
        return(ConcatableAutomaton(TransitionList))
        

    def odd_accepter_machine(self) -> ConcatableAutomaton:
        '''
        This machine will accept words of odd length. Since words will be passed as lists of elements of the alphabet, the len function could do this.
        However, the tools in sage.combinat.finite_state_machine don't easily facilitate combining FSMs with functions.

        :return: An automaton whose accepted language consists of (unreduced) words of odd length.
        '''

        EvenState = FSMState('even', is_initial = True, is_final = False)
        OddState = FSMState('odd', is_initial = False, is_final = True)
        TransitionList = []
        for letter in self.alphabet:
            TransitionList.append(FSMTransition(EvenState, OddState, letter))
            TransitionList.append(FSMTransition(OddState, EvenState, letter))
        return ConcatableAutomaton(TransitionList)

    def even_accepter_machine(self) -> ConcatableAutomaton:
        '''
        This machine will accept words of even length. Since words will be passed as lists of elements of the alphabet, the len function could do this.
        However, the tools in sage.combinat.finite_state_machine don't easily facilitate combining FSMs with functions.

        :return: An automaton whose accepted language consists of (unreduced) words of odd length.
        '''

        EvenState = FSMState('even', is_initial = True, is_final = True)
        OddState = FSMState('odd', is_initial = False, is_final = False)
        TransitionList = []
        for letter in self.alphabet:
            TransitionList.append(FSMTransition(EvenState, OddState, letter))
            TransitionList.append(FSMTransition(OddState, EvenState, letter))
        return ConcatableAutomaton(TransitionList)
        
    def __shortlex_machine(self, restricted_alphabet = None) -> ConcatableAutomaton:
        """
        Generate an FSM that prevents letters from being written that either cancel or should have already been written. 
        The states of this automaton represent the list of letters that a word cannot be followed by if it is to remain shortlex

        :param restricted_alphabet: if desired, pass a subset of self.alphabet to get the shortlex machine for the special subgroup defined by the vertices in restricted_alphabet. If this set is empty, an FSM recognizing the empty word is returned.
        
        :return: An automaton whose accepted language consists of shortlex words.
        """
        if restricted_alphabet is None:
            restricted_alphabet = self.alphabet
        if not restricted_alphabet.issubset(self.alphabet):
            raise ValueError("argument restricted_alphabet is not a subset of self.alphabet")
        if restricted_alphabet == set():
            print("You have requested the shortlex machine on an empty alphabet. Returning an automaton that accepts only the empty string")
            SingleState = FSMState('origin', is_initial = True, is_final = True)
            return ConcatableAutomaton({SingleState:[]})
        
        StartState = FSMState((), is_initial = True, is_final = True)
        TransitionList = []
        Frontier = []
        States = [StartState]

        # Initialize the frontier with the legal next letters for single letter words 
        for nextletter in restricted_alphabet:

            # Set destination as the set of legal next letters for each letter
            NextName = tuple(sorted((self.lesser_star[nextletter].intersection(restricted_alphabet)).union({nextletter}), key = lambda x: self.o_map[x]))
            NextState = FSMState(NextName, is_initial = False, is_final = True)
            TransitionList.append(FSMTransition(StartState, NextState, nextletter))
            
            Frontier.append(NextState)

        # Run BFS until we have finished the machine
        while len(Frontier) > 0:
            SourceState = Frontier.pop(0)

            # We have already considered source and all its outgoing edges, we can skip it
            if SourceState in States:
                continue
            States.append(SourceState)

            # Record outgoing edges (i.e. possible next letters) from SourceState. This is guaranteed to be unique by above if-statement
            SourceSet = set(SourceState.label())
            for nextletter in restricted_alphabet.difference(SourceSet):
                # This line computes the new set of forbidden letters
                NextSet = SourceSet.intersection(self.c_map[nextletter]).union(self.lesser_star[nextletter].intersection(restricted_alphabet)).union({nextletter})
                NextName = tuple(sorted(NextSet, key = lambda x: self.o_map[x]))
                #We can never return to the original state, so the next state is always final and never initial.
                NextState = FSMState(NextName, is_initial = False, is_final = True)
                                
                # Add NextState to our frontier to ensure all vertices are reached
                Frontier.append(NextState)
                TransitionList.append(FSMTransition (SourceState, NextState, nextletter))
               
        #print(f"ShortLex Machine on alphabet {restricted_alphabet} Completed: Graph with \n\t\t{len(States)} Vertices and \n\t\t{len(TransitionList)} Edges.")
        return (ConcatableAutomaton(TransitionList))
    
    def __geodesic_machine(self, restricted_alphabet = None)->ConcatableAutomaton:
        """
        Generate an FSM that prevents letters from being written that cancel . 
        The states of this automaton represent the list of letters that a word cannot be followed by if it is to remain geodesic

        :param restricted_alphabet: if desired, pass a subset of self.alphabet to get the geodesic machine for the special subgroup defined by the vertices in restricted_alphabet.
        
        :return: An automaton whose accepted language consists of geodesic words.
        """
        if restricted_alphabet is None:
            restricted_alphabet = self.alphabet
        if not restricted_alphabet.issubset(self.alphabet):
            raise ValueError("argument restricted_alphabet is not a subset of self.alphabet")
        if restricted_alphabet == set():
            print("You have requested the geodesic machine on an empty alphabet. Returning an automaton that accepts only the empty string")
            SingleState = FSMState('origin', is_initial = True, is_final = True)
            return ConcatableAutomaton({SingleState:[]})
        
        StartState = FSMState((), is_initial = True, is_final = True)
        TransitionList = []
        Frontier = []
        States = [StartState]

        # Initialize the frontier with the legal next letters for single letter words 
        for nextletter in restricted_alphabet:

            # Set destination as the set of legal next letters for each letter
            NextName = (nextletter)
            NextState = FSMState(NextName, is_initial = False, is_final = True)
            TransitionList.append(FSMTransition(StartState, NextState, nextletter))
            
            Frontier.append(NextState)

        # Run BFS until we have finished the machine
        while len(Frontier) > 0:
            SourceState = Frontier.pop(0)

            # We have already considered source and all its outgoing edges, we can skip it
            if SourceState in States:
                continue
            States.append(SourceState)

            # Record outgoing edges (i.e. possible next letters) from SourceState. This is guaranteed to be unique by above if-statement
            SourceSet = set(SourceState.label())
            for nextletter in restricted_alphabet.difference(SourceSet):
                # This line computes the new set of forbidden letters
                NextSet = SourceSet.intersection(self.c_map[nextletter]).union({nextletter})
                NextName = tuple(sorted(NextSet, key = lambda x: self.o_map[x]))
                #We can never return to the original state, so the next state is always final and never initial.
                NextState = FSMState(NextName, is_initial = False, is_final = True)
                                
                # Add NextState to our frontier to ensure all vertices are reached
                Frontier.append(NextState)
                TransitionList.append(FSMTransition (SourceState, NextState, nextletter))
               
        #print(f"Geodesic Machine on alphabet {restricted_alphabet} completed: Graph with \n\t\t{len(States)} Vertices and \n\t\t{len(TransitionList)} Edges.")
        return (ConcatableAutomaton(TransitionList))
    
    def shortlex_suffix_machine(self) -> ConcatableAutomaton:
        return self.__shortlex_machine().concatable_intersection(self.__first_letter_excluder(set(self.ray)))

    def geodesic_suffix_machine(self) -> ConcatableAutomaton:
        return self.__geodesic_machine().concatable_intersection(self.__first_letter_excluder(set(self.ray)))



In [25]:
class Divergence_FSM_Generator(Rips_FSM_Generator):
    def __init__(self, commutation_dict:dict[str, set], order_dict: dict[str, int], ray: list[str]):
        
        '''
        This object will make the necessary automata for computing the Divergence Graph. 
    
        :param commutation_dict: A dictionary representation of a defining graph. A letter (key) is associated with a list of letters (value) that the key letter commutes with. This list should not include the key letter.
        Note: this program does not check that commutation_dict is symmetrical. You need to make sure that if your input allows a_i to commute with a_j, then a_j is also allowed to commute with a_i.
        :param order_dict: A dictionary representation of a total ordering on the letter in the defining graph. A letter (key) is associated with an index (value) that represents that letters relative position in the ordering.
        :param ray: A list of two characters that will be used to generate the chosen ray to infinity. In the paper these are referred to as a_i and a_j respectively.
        '''

        if not len(ray) == 2:
            raise ValueError ("The defining ray should have two letters")
        elif ray [1] in commutation_dict[ray[0]]:
            raise ValueError ("The defining ray consists of two letters which commute")
        
        self.c_map = commutation_dict
        self.o_map = order_dict
        self.alphabet = set().union(letter for letter in self.o_map)
        self.ray = ray

        self.WordGeneratorMachine = WordGenerator(self.c_map, self.o_map)
        
        StringOfAllLetters = ''.join(letter for letter in self.alphabet)
        if '-' in StringOfAllLetters:
            raise ValueError ('''The character '-' is reserved to be the blank character''')
        if ',' in StringOfAllLetters:
            raise ValueError('''The character ',' is reserved for delimiting lists''')
        if '_' in StringOfAllLetters:
            raise ValueError(''' The character '_' is reserved for the word breaks in horocyclic suffices''')
        
        #the lesser_star dictionary agrees with the function Star_< in the paper. When passed a key of a letter, it returns the set of letters that commute with and precede the key. Note the strict inequality.
        self.lesser_star = {}
        for letter in self.alphabet:
            LesserStarOfLetter = set(filter(lambda x: self.o_map[x] < self.o_map[letter], self.c_map[letter]))
            self.lesser_star[letter] = LesserStarOfLetter
        self.greater_star = {}
        for letter in self.alphabet:
            self.greater_star[letter] = self.c_map[letter].difference(self.lesser_star[letter])

    def horocyclic_suffix_machine_1(self) -> ConcatableAutomaton:
        #The w_1 machine accepts words spelled in the letters that commute with both letters of the ray and precede the second one (a_j in the paper)
        w1_machine = self._Rips_FSM_Generator__shortlex_machine(self.c_map[self.ray[0]].intersection(self.lesser_star[self.ray[1]]))

        return (w1_machine)

    def horocyclic_suffix_machine_2(self) -> ConcatableAutomaton:
        #The w_2 machine accepts words spelled in the letters that commute with both letters of the ray, precede the first one (a_i in the paper), and follow the second one.
        w2_machine = self._Rips_FSM_Generator__shortlex_machine(self.lesser_star[self.ray[0]].intersection(self.greater_star[self.ray[1]]))

        return(w2_machine)

    
    def horocyclic_suffix_machine_1234(self) -> ConcatableAutomaton:
        '''
        Generate an FSM which accepts one of the two horocyclically shortlex word forms.

        :return: an Automaton whose accepted language is the set of words w_1w_2w_3w_4 as described in the paper.
        ''' 

        #w_3w_4 is a concattenation of shortlex words such that
        # 1. w_3 is spelled with letters commuting with and preceding a_j, and which do not commute with a_i
        # 2. w_4 cannot be made to begin with a_i, a_j, or a letter commuting with and preceding a_j
        # 3. w_3w_4, as a whole, cannot be rearranged to begin with a letter commuting with both letters and preceding a_i

        w3_machine = self._Rips_FSM_Generator__shortlex_machine(self.lesser_star[self.ray[1]]).concatable_intersection(self._Rips_FSM_Generator__first_letter_excluder(self.c_map[self.ray[0]]))

        #This machine is so-named because its language is not exactly the words w_4, since in fact whether a word is allowed to be w_4 depends on w_3 by condition 3 above
        w4_machine_approx = self._Rips_FSM_Generator__shortlex_machine().concatable_intersection(self._Rips_FSM_Generator__first_letter_excluder(self.lesser_star[self.ray[1]].union(set(self.ray))))
        #If there are bugs, it's because of this machine, but I believe this machine is correct.

        w1_machine = self.horocyclic_suffix_machine_1()
        w2_machine = self.horocyclic_suffix_machine_2()
        w12_machine = w1_machine.unambiguous_concatenation(w2_machine)
        
        w3w4_machine = (w3_machine.unambiguous_concatenation(w4_machine_approx)).concatable_intersection(self._Rips_FSM_Generator__first_letter_excluder({self.ray[0]}.union(self.lesser_star[self.ray[0]].intersection(self.c_map[self.ray[1]]))))

        return w12_machine.unambiguous_concatenation(w3w4_machine)

    def horocyclic_suffix_machine_1256(self) -> ConcatableAutomaton:
  
        '''
        Generate an FSM which accepts one of the two horocyclically shortlex word forms.

        :return: an Automaton whose accepted language is the set of words w_1w_2w_5w_6 as described in the paper.
        ''' 

        #w_5w_6 is a concattenation of shortlex words such that
        # 1. w_5 is spelled with letters commuting with and preceding a_i, and which do not commute with a_j
        # 2. w_6 cannot be made to begin with a_i, a_j, or a letter commuting with and preceding a_i
        # 3. w_5w_6, as a whole, cannot be rearranged to begin with a letter commuting with both a_i and a_j and preceding a_j
        
        w5_machine = self._Rips_FSM_Generator__shortlex_machine(self.lesser_star[self.ray[0]]).concatable_intersection(self._Rips_FSM_Generator__first_letter_excluder(self.c_map[self.ray[1]]))

        #This machine is so-named because its language is not exactly the words w_6, since in fact whether a word is allowed to be w_6 depends on w_5 by condition 3 above
        w6_machine_approx = self._Rips_FSM_Generator__shortlex_machine().concatable_intersection(self._Rips_FSM_Generator__first_letter_excluder(self.lesser_star[self.ray[0]].union(set(self.ray))))
        #If there are bugs, it's because of this machine, but I believe this machine is correct.

        w1_machine = self.horocyclic_suffix_machine_1()
        w2_machine = self.horocyclic_suffix_machine_2()
        w12_machine = w1_machine.unambiguous_concatenation(w2_machine)
        
        w5w6_machine = (w5_machine.unambiguous_concatenation(w6_machine_approx)).concatable_intersection(self._Rips_FSM_Generator__first_letter_excluder({self.ray[1]}.union(self.lesser_star[self.ray[1]].intersection(self.c_map[self.ray[0]]))))

        return w12_machine.unambiguous_concatenation(w5w6_machine)

    #We will not use the even_horocyclic_suffix_machine or the odd_horocyclic_suffix_machine in practice.
    
    def even_horocyclic_suffix_machine(self) -> ConcatableAutomaton:

        return ((self.horocyclic_suffix_machine_1256())._interspersal(self.horocyclic_suffix_machine_1234()))

    def odd_horocyclic_suffix_machine(self) -> ConcatableAutomaton:

        return ((self.horocyclic_suffix_machine_1234())._interspersal(self.horocyclic_suffix_machine_1256()))

    def horocyclic_edge_checker(self, SubwordDict) -> Automaton:
        '''
        This machine will check whether there is a divergence graph edge between two horocyclic suffixes of the same length.
        The inputs will be pairs (WLetter,VLetter) or the pair ('-','-'). The latter denotes the end of the word.

        The subroutines generate_states_without_uncancelables, generate_first_noncanceling_transitions, get_nonterminal_transitions,
        and get_double_blank_transition will all be defined later.
        
        :param SubwordDict: A dictionary that keeps track of which letters first appear in which subword. It will have values 1-4 if the two words are of the same length and values 1-3 if the two are of different lengths.
        This is because in the case that w_suff is 1 letter shorter than v_suff, we will compare w_1w_2w_3self.ray[1]w_4- with v_1v_2 v_6-, or w_1w_2w_5 self.ray[0]w_6- with v_1v_2 v_4-.
        In the first case, letters of w_3self.ray[1]w_4 can cancel with those of v_6, while in the second case, letters of w_5self.ray[0]w_6 can cancel with those of v_4.
        
        :return: an Automaton which accepts as inputs pairs of letters, one from w and one from v, such that the words such that the lengths of each word are equal and there is no uncancelable pair
        '''

        
        

        NonFinalStates = []
        FinalStates = []
        TotalTransitions = []
        FinishedStates = []

        FinalSubword = max(SubwordDict.values())

        (StatesWithoutUncancelables, NewTransitions) = self.generate_states_without_uncancelables(SubwordDict)

        print('Completed the uncancelable states. There are ', len(StatesWithoutUncancelables), ' of them.')
        
        NonFinalStates.extend(StatesWithoutUncancelables)
        TotalTransitions.extend(NewTransitions)
        
        for StateWithoutUncancelables in StatesWithoutUncancelables:
            (NewStates, NewTransitions) = self.generate_first_noncanceling_transitions(StateWithoutUncancelables,SubwordDict)
            NonFinalStates.extend(NewStates)
            TotalTransitions.extend(NewTransitions)
            FinishedStates.append(StateWithoutUncancelables)

        print('Completed the first batch of transitions. There are ', len(TotalTransitions), 'transitions and ', len(set(NonFinalStates)), ' states in total')
        
        while len(NonFinalStates)>0:
            SourceState = NonFinalStates.pop(0)
            if SourceState in FinishedStates:
                continue

            (NewStates, NewTransitions) = self.get_nonterminal_transitions(SourceState, SubwordDict)
            NonFinalStates.extend(NewStates)
            TotalTransitions.extend(NewTransitions)
            FinishedStates.append(SourceState)

        print('Completed the nonterminal transitions. There are ', len(TotalTransitions), ' transitions and ', len(set(FinishedStates)), ' states in total')
        
        for SourceState in FinishedStates:
            try:
                (FinalState, FinalTransition) = self.get_double_blank_transition(SourceState, FinalSubword)
            except TypeError:
                continue
            FinalStates.append(FinalState)
            TotalTransitions.append(FinalTransition)

        print('completed the final transitions. There are ', len(TotalTransitions), 'transitions and ', len(set(FinalStates)), ' final states in total')
        
        return Automaton(TotalTransitions)

    def horocyclic_edge_checker_different_length(self) -> Automaton:
        '''
        This machine will check whether there is an uncancelable pair between two horocyclic suffixes of length differing by 1. The first input is w and the second is v.
        v_3 or v_5, depending on the form of v, is necessarily empty to have such an edge. However, this machine will not check this fact explicitly.
        The inputs will be pairs (WLetter, VLetter) or the pair ('-', '-'). The latter denotes the end of the word.
        
        The subroutines generate_states_without_uncancelables, generate_first_noncanceling_transitions, get_nonterminal_transitions,
        and get_single_blank_transition will all be defined later.
        
        :return: an Automaton which accepts pairs (w_1w_2w_3self.ray[1]w_4-, v_1v_2 v_6-) and (w_1w_2w_4 self.ray[0]w_6-, v_1v_2 v_4-) such that the lengths of each word are equal and there is no uncancelable pair
        '''

        #if not (mode == '1234' or mode == '1256'):
            #raise ValueError(''' The mode should be either '1234' or '1256' ''')

        #This dictionary keeps track of the necessary changes to the parameters n_v and n_w
        SubwordDict = {}
        #The letters that commute with and precede both ray letters appear first in w_1
        for letter in self.c_map[self.ray[0]].intersection(self.lesser_star[self.ray[1]]):
            SubwordDict[letter] = 1
        #The letters that commute with both ray letters and precede only the first appear first in w_2
        for letter in self.lesser_star[self.ray[0]].intersection(self.greater_star[self.ray[1]]):
            SubwordDict[letter] = 2
        #Every other letter appears in one of the last two subwords, and these are all lumped into a single case.
        for letter in self.alphabet:
            try:
                SubwordDict[letter]
            except KeyError:
                SubwordDict[letter] = 3

        NonFinalStates = []
        PreFinalStates = []
        FinalStates = []
        TotalTransitions = []
        FinishedStates = []

        #The beginning of this machine is the same as the previous one. All that needs to change is that we need to check that there are precisely two input pairs ('-', 'letter').
        #Because of these two blank characters at the end of the word w, there is no need for an input ('-', '-') to indicate that the word has ended.
        
        (StatesWithoutUncancelables, NewTransitions) = self.generate_states_without_uncancelables(SubwordDict)

        print('Completed the uncancelable states. There are ', len(StatesWithoutUncancelables), ' of them.')
        
        NonFinalStates.extend(StatesWithoutUncancelables)
        TotalTransitions.extend(NewTransitions)
        
        for StateWithoutUncancelables in StatesWithoutUncancelables:
            (NewStates, NewTransitions) = self.generate_first_noncanceling_transitions(StateWithoutUncancelables,SubwordDict)
            NonFinalStates.extend(NewStates)
            TotalTransitions.extend(NewTransitions)
            FinishedStates.append(StateWithoutUncancelables)

        print('Completed the first batch of transitions. There are ', len(TotalTransitions), 'transitions and ', len(NonFinalStates), ' states in total')
        
        while len(NonFinalStates) > 0:
            SourceState = NonFinalStates.pop(0)
            if SourceState in FinishedStates:
                continue

            (NewStates, NewTransitions) = self.get_nonterminal_transitions(SourceState, SubwordDict)
            NonFinalStates.extend(NewStates)
            TotalTransitions.extend(NewTransitions)
            FinishedStates.append(SourceState)

        print('Completed the nonterminal transitions. There are ', len(TotalTransitions), ' transitions and ', len(set(FinishedStates)), ' states in total')
        
        for SourceState in set(FinishedStates):
            for letter in SourceState.label()[9]:
                holder = self.get_single_blank_transition(SourceState, letter, SubwordDict)
                if holder is None:
                    continue
                (PreFinalState, PreFinalTransition) = holder
                PreFinalStates.append(PreFinalState)
                TotalTransitions.append(PreFinalTransition)

        print('Completed the first set of blank transitions. There are ', len(TotalTransitions), 'transitions and ', len(set(PreFinalStates)), ' pre-final states in total')

        for SourceState in set(PreFinalStates):
            for letter in SourceState.label()[9]:
                holder = self.get_single_blank_transition(SourceState, letter, SubwordDict)
                if holder is None:
                    continue
                (FinalState, FinalTransition) = holder
                FinalStates.append(FinalState)
                TotalTransitions.append(FinalTransition)

        print('Completed the second set of blank transitions. There are ', len(TotalTransitions), 'transitions and ', len(set(FinalStates)), ' final states in total')
        
        return Automaton(TotalTransitions)
        
    '''
    What follow are subroutines for the methods horocyclic_edge_checker_same_length and horocyclic_edge_checker_different_length.
    To improve performance and make certain features of sage.combinat.finite_state_machine usable, the state labels must be hashable, but to make changes, we will want mutable types.

    The following describes the state labels
    4 entries for the subwords u_j (j=1, 2, 3, 4) of potentially cancelable letters. These will be tuples (hashable) or instances of the Word Class (mutable)
    4 entries for the geodesic first letters of each of the u_j, and of each of their truncations. These will be tuples of pairs of tuples (hashable) or lists of pairs of sets (mutable)
    This will be formatted as a tuple or list of pairs (first letters, potential first letters).
    These first 8 entries will alternate, so that the data for the subword u_j appears at indices 2*j-2 and 2*j-1.
    1 entry for the uncancelable letters, in a tuple (hashable) or set (mutable).
    1 entry for the letters that commute with each uncancelable letter, in a tuple (hashable) or set (mutable)
    1 tuple of 2 integers n_w and n_v keeping track of which subword the two inputs are on.
    1 bit. True will mean that the second input (v) is the one which contains the potentially cancelable letters. False will mean that the first input (w) contains the potentially cancelable letters. Its initial value is arbitrarily set to True.
    The two inputs are the 'adding input' (the word that has the cancelable letters) and the 'canceling input'. So the bit True means that v is adding and w is canceling

    All of the above data will be placed into a tuple (hashable) or list (mutable)
    '''
    
    def generate_states_without_uncancelables(self, SubwordDict:dict[str: int]) -> tuple:
        #There are at most 4 states without uncancelable letters. This method generates them and the transitions between them.
        #:param SubwordDict: A dictionary whose keys are letters and whose values are the first subword that letter appears in.
        #:return: A pair consisting of a list of states without uncancelable letters, and a list of the transitions between them.
        
        TransitionList = []
        StatesWithoutUncancelables = []

        #This state will always exist
        InitialState = FSMState( ( (), (), (),  (), (), (), (), (), (), tuple(self.alphabet), (1, 1), True ),
                              is_initial = True, is_final = False)

        StatesWithoutUncancelables.append(InitialState)

        #We only need a state without uncancelables for each subword that will actually appear.
        for value in set(SubwordDict.values()).difference({1}):
            NewState = FSMState( ( (), (), (),  (), (), (), (), (), (), tuple(self.alphabet), (value, value), True ) )
            StatesWithoutUncancelables.append(NewState)
               
        for SourceState in StatesWithoutUncancelables:
            for letter in self.alphabet:
                NewSubword = max(SourceState.label()[10][0], SubwordDict[letter])
                NewState = FSMState( ( (), (), (),  (), (), (), (), (), (), tuple(self.alphabet), (NewSubword, NewSubword), True ) ,
                              is_initial = (NewSubword == 1), is_final = False)
                TransitionList.append(FSMTransition(SourceState, NewState, (letter, letter)))

        return (StatesWithoutUncancelables, TransitionList)

    def generate_first_noncanceling_transitions(self, StateWithoutUncancelables:FSMState, SubwordDict:dict[str: int]) -> tuple:
        #This method generates all the remaining transitions out of the 4 states without uncancelables
        #:param StateWithoutUncancelables: An FSMState that is in the output state list of FSM_Generator.generate_states_without_uncancelables.
        #:param SubwordDict: A dictionary whose keys are letters and whose values are the first subword that letter appears in.
        #:return: A pair consisting of a list of the states that follow StateWithoutUncancelables and a list a of the transitions out of StateWithoutUncancelables.

        if StateWithoutUncancelables.label()[8] != ():
            raise ValueError('This state has uncancelable letters')
        
        ResultingStates = []
        Transitions = []
        
        for LetterPair in product(self.alphabet, self.alphabet):
            WLetter = LetterPair [0]
            VLetter = LetterPair [1]
            #We can assume the two commute, or else we immediately reach a failure state
            if WLetter not in self.c_map[VLetter]:
                #This includes the case that WLetter == VLetter
                continue

            #In this setting, we will need to change the value at every 
            NewLabel = self._mutable_label(StateWithoutUncancelables.label())
            NewNW = max(NewLabel[10][0], SubwordDict[WLetter])                    
            NewNV = max(NewLabel[10][1], SubwordDict[VLetter])
            NewLabel[10] = (NewNW, NewNV)
            #We compute the next value of the bit. True means that the v is the adding word and False means that w is the adding word.
            NewBit = ((NewNW < NewNV) or (NewNW == NewNV and self.o_map[WLetter] < self.o_map[VLetter]))
            NewLabel[11] = NewBit
            AddingLetter = LetterPair[int(NewBit)]
            AddingSubword = NewLabel[10][int(NewBit)]
            CancelingLetter = LetterPair[1-int(NewBit)]
            
            #Add the AddingLetter
            NewLabel[2*AddingSubword -2].append(AddingLetter)
            #The AddingLetter is the first letter of the relevant word. The other possible first letters are those that commute with it.
            NewLabel[2*AddingSubword -1].append( ({AddingLetter}, self.c_map[AddingLetter]) )
            
            #Since the Adding and Canceling letters do not equate, there is no need to check for cancelation. The CancelingLetter automatically becomes uncancelable.
            NewLabel[8] = {CancelingLetter}
            NewLabel[9].intersection_update(self.c_map[CancelingLetter])
            
            NewState = FSMState(self._hashable_label(NewLabel))
            ResultingStates.append(NewState)
            NewTransition = FSMTransition(StateWithoutUncancelables, NewState, LetterPair)
            Transitions.append(NewTransition)
        return(ResultingStates, Transitions)

    def get_nonterminal_transitions(self, SourceState:FSMState, SubwordDict: dict[str: int]) -> tuple:
        #This method generates all the nonterminal successors of a state that has some uncancelable letters
        #:param SourceState: An FSMState with some uncancelable letters. 
        #:param SubwordDict: A dictionary whose keys are letters and whose values are the first subword that letter appears in.
        #:return: A pair consisting of a list of the states that follow SourceState and a list a of the transitions out of SourceState.

        OldBit = SourceState.label()[11]
        (OldNW, OldNV) = SourceState.label()[10]
        OldSubwordPair = (OldNW, OldNV)

        ResultingStates = []
        Transitions = []

        for LetterPair in product(SourceState.label()[9], SourceState.label()[9]):
            WLetter = LetterPair [0]
            VLetter = LetterPair [1]
            AddingLetter = LetterPair[int(OldBit)]
            CancelingLetter = LetterPair[1-int(OldBit)]
            
            NewLabel = self._mutable_label(SourceState.label())
            NewNW = max(NewLabel[10][0], SubwordDict[WLetter])                    
            NewNV = max(NewLabel[10][1], SubwordDict[VLetter])
            NewSubwordPair = (NewNW, NewNV)
            AddingSubword = NewSubwordPair[int(OldBit)]
            CancelingSubword = NewSubwordPair[1-int(OldBit)]
            NewLabel[10] = NewSubwordPair

            #Append the new letter to the relevant subword
            NewLabel[2*AddingSubword-2].append(AddingLetter)
            #Update the first leters
            for FirstLettersSet in NewLabel[2*AddingSubword-1]:
                if AddingLetter in FirstLettersSet[1]:
                    FirstLettersSet[0].add(AddingLetter)
                FirstLettersSet[1].intersection_update(self.c_map[AddingLetter])
            NewLabel[2*AddingSubword-1].append( ({AddingLetter},self.c_map[AddingLetter]) )

            #Compute the next bit value
            PresentLetterList = NewLabel[0].word_as_list + NewLabel[2].word_as_list + NewLabel[4].word_as_list + NewLabel[6].word_as_list
            PresentLetterSet = set(PresentLetterList)
            #Check whether the bit value needs to be flipped
            LettersFollowingAllPresentLetters = copy.copy(self.alphabet)
            for letter in PresentLetterSet:
                LettersFollowingAllPresentLetters.intersection_update(self.greater_star[letter])
            BitFlip = ( (CancelingSubword > AddingSubword) or \
                       (CancelingSubword == AddingSubword) and CancelingLetter in LettersFollowingAllPresentLetters \
                      )
            #exor True is the same as not, while exor False does nothing.
            NewLabel[11] = OldBit ^ BitFlip

            if BitFlip:
                #Every previously cancelable letter will become uncancelable. 
                #This will create an uncancelable pair if any of them fail to cancel with one another or with the CancelingLetter.
                #We also check whether there are any duplicates among the PresentLetters.
                #It is not possible for the CancelingLetter to be in PresentLetters in this case, so we need not check for disjointness.
                if (len(PresentLetterList) > len(PresentLetterSet)) or (not (self._test_set_commutation(PresentLetterSet) and self._test_set_pair_commutation({CancelingLetter},PresentLetterSet))):
                    continue
                                
                #Add every remaining cancelable letter into the uncancelable set
                NewLabel[8] = NewLabel[8].union(PresentLetterSet)
                for letter in PresentLetterSet:
                    NewLabel[9].intersection_update(self.c_map[letter])
            
                #Delete every previously cancelable letter, and initialize a new single cancelable letter.
                #Because we have just flipped the bit, the NewCancelingSubword and CancelingLetter take the place that the NewAddingSubword and AddingLetter had
                
                for i in range(0,4):
                    NewLabel[2*i] = self.WordGeneratorMachine.word([])
                    NewLabel[2*i+1] = []
                NewLabel[2*CancelingSubword-2] = self.WordGeneratorMachine.word(CancelingLetter)
                NewLabel[2*CancelingSubword-1].append( ({CancelingLetter}, self.c_map[CancelingLetter]) )
                NewState = FSMState(self._hashable_label(NewLabel))
                ResultingStates.append(NewState)
                NewTransition = FSMTransition(SourceState, NewState, LetterPair)
                Transitions.append(NewTransition)
                continue

            #Since the bit has not flipped, we just need to process the canceling letter.
            NewLabel = self.process_canceling_letter(NewLabel, CancelingLetter, SubwordDict)
            if NewLabel is None:
                continue
            NewState = FSMState(self._hashable_label(NewLabel))
            ResultingStates.append(NewState)
            NewTransition = FSMTransition(SourceState, NewState, (WLetter, VLetter))
            Transitions.append(NewTransition)

        return (ResultingStates, Transitions)

    def process_canceling_letter(self, Label:list, CancelingLetter:str, SubwordDict:dict[str: int])-> list:
        #This method takes a mutable label of an FSMState to which the AddingLetter has already been added and for which the bit has not flipped, and evaluates whether the CancelingLetter cancels
        #This method updates the label to change the cancelable letters, uncancelable set, and list of accepted next letters appropriately.
        #:param Label: the mutable label of an FSMState.
        #:param CancelingLetter: the letter whose cancelation is to be tested.
        #:param SubwordDict: A dictionary whose keys are letters and whose values are the first subword that letter appears in.
        #:return: The mutable label of the resulting state, or None if processing CancelingLetter creates an uncancelable pair.

        CancelingSubword = Label[10][1-int(Label[11])]
        NewUncancelableList = []
        #If a new subword has started, then the remaining letters from the previous subword(s) become uncancellable
        for i in range (0, CancelingSubword-1):
            NewUncancelableList.extend(Label[2*i].word_as_list)
            Label[2*i] = self.WordGeneratorMachine.word([])
            Label[2*i+1] = []
        NewUncancelables = set(NewUncancelableList)
        #This will create an uncancellable pair if any of the newly uncancelable letters fail to commute with one another, or any of them fail to commute with a remaining letter, or if they don't commute with the CancelingLetter
        #Since there may be more NewUncancelables after checking whether the CancelingLetter cancels all we must check now is whether the CancelingLetter commutes to the matching AddingSubword 
        #It is not possible for CancelingLetter to be in NewUncancelables because because then CancelingLetter would have to have appeared in the adding subword earlier than possible. So there is not need to ask for disjointness
        if not self._test_set_pair_commutation({CancelingLetter}, NewUncancelables):
            return (None)

        #Now we check whether the CancelingLetter actually cancels.
        #This idiom avoids problems with indexing into empty lists.
        if CancelingLetter in next(iter(Label[2*CancelingSubword-1]), (set(), set()))[0]:
            CancelingIndex = Label[2*CancelingSubword-2].index(CancelingLetter)
            #These letters have just become uncancellable.
            NewUncancelableList.extend(Label[2*CancelingSubword-2][:CancelingIndex])
            #This is the remaining potentially cancelable word.
            Label[2*CancelingSubword-2] = self.WordGeneratorMachine.word(Label[2*CancelingSubword-2][CancelingIndex+1:])
            Label[2*CancelingSubword-1] = Label[2*CancelingSubword-1][CancelingIndex+1:]
            #Update which letters are present
            RemainingLetters = set(Label[0].word_as_list).union(set(Label[2].word_as_list),set(Label[4].word_as_list),set(Label[6].word_as_list))
            #Check whether there are duplicate uncancellable letters
            NewUncancelables = set(NewUncancelableList)
            if len(NewUncancelableList) > len(NewUncancelables):
                return(None)
            #Check whether there is an uncancellable pair, either between the new uncancelables or with the remaining letters. In this case we will demand disjointness.
            if not (self._test_set_pair_commutation(NewUncancelables , RemainingLetters, AllDistinct = True) and self._test_set_commutation(NewUncancelables)):
                return(None)
            #If there are no uncancelable pairs, then we get a new transition. We will update the uncancelable set and the set of accepted next letters outside the if statement.
            for letter in NewUncancelables:
                Label[8].add(letter)
                Label[9].intersection_update(self.c_map[letter])
            return(Label)
        else:
            #If the CancelingLetter does not cancel, then it joins the uncancelable set. We check whether this creates an uncancelable pair.
            NewUncancelables.add(CancelingLetter)
            RemainingLetters = set(Label[0].word_as_list).union(set(Label[2].word_as_list),set(Label[4].word_as_list),set(Label[6].word_as_list))
            if not (self._test_set_pair_commutation(NewUncancelables, RemainingLetters, True) and self._test_set_commutation(NewUncancelables)):
                return(None)
            #If there are no uncancelable pairs, then we get a new transition.
            for letter in NewUncancelables:
                Label[8].add(letter)
                Label[9].intersection_update(self.c_map[letter])
            return(Label)
    
    def get_double_blank_transition(self, SourceState:FSMState, FinalSubword:int)->tuple:
        #This method takes any nonterminal SourceState and computes the outgoing transition labeled ('-', '-').
        #:param SourceState: a non-final FSMState.
        #:param FinalSubword: 3 or 4, depending on the number of different values of SubwordDict.
        #:return: A pair consisting of a final state and a transition from SourceState to this final state labeled is ('-', '-'), or None if ending the inputs here yields an uncancelable pair.

        if SourceState.is_final:
            raise ValueError('The provided state is final already')
        NewLabel = self._mutable_label(SourceState.label())
        NewUncancelableList = []
        for i in range (1, FinalSubword):
            NewUncancelableList.extend(NewLabel[2*i-2].word_as_list)
            NewLabel[2*i-2] = self.WordGeneratorMachine.word([])
            NewLabel[2*i-1] = []
        PresentLetters = set(NewLabel[2*FinalSubword - 2].word_as_list)
        NewUncancelables = set(NewUncancelableList)
        #Checking whether there are duplicate uncancelable letters.
        if len(NewUncancelableList) > len(NewUncancelables):
            return(None)
        if not (self._test_set_commutation(NewUncancelables) and self._test_set_pair_commutation(NewUncancelables, PresentLetters, True)):
            return(None)
        for letter in NewUncancelables:
            NewLabel[8].add(letter)
            NewLabel[9].intersection_update(self.c_map[letter])
        NewLabel[10] = (FinalSubword, FinalSubword)
        
        NewLabel.append('final')
        #Necessary to prevent issues with states that have matching labels.
        
        NewState = FSMState(self._hashable_label(NewLabel), is_initial = False, is_final = True)
        return( NewState, FSMTransition (SourceState, NewState, ('-','-')) )

    '''
    For the case where the words are different length, we will need one more subroutine. Since the words are of different lengths, we will assume the first input w is the shorter word.
    Then we will pad the input so that w ends with 2 blank characters '-' so that w-- and v_1v_2v_3 self.ray[1] v_4 (or v_1v_2v_5 self.ray[0] v_6) are the same length
    Therefore, we give a subroutine to process input pairs ('-', letter).
    '''

    def get_single_blank_transition(self, SourceState: FSMState, InputLetter: str, SubwordDict: dict[str:int]) -> tuple:
        '''
        This method takes a SourceState and returns the result of a transition labeled ('-', 'InputLetter').
    
        :param SourceState: an instance of FSMState of the above form.
        :param InputLetter: The input letter of the subword v. This may be canceling or adding.
        :param SubwordDict: A dictionary whose keys are letters and whose values are the first subword that letter appears in.
    
        :return: a pair (NewState, NewTransition) if the input ('-', InputLetter) does not create an uncancelable pair. 
        NewTransition is an instance of FSMTransition whose from_state is SourceState and whose to_state is NewState.
        If an uncancelable pair is created, this method returns None.
        '''
        
        NewLabel = self._mutable_label(SourceState.label())
        NewUncancelableList = []
    
    
        #Subwords w_1 and w_2 have ended.
        NewNV = max(NewLabel[10][1], SubwordDict[InputLetter])
        NewLabel[10] = (3, NewNV)
        
        #We will process this input in two cases depending on whether InputLetter is adding or cancelling. The adding case is easiest.
    
        if NewLabel[11]:
            NewLabel[2*NewNV-2].append(InputLetter)
            for FirstLettersSet in NewLabel[2*NewNV-1]:
                if InputLetter in FirstLettersSet[1]:
                    FirstLettersSet[0].add(InputLetter)
                FirstLettersSet[1].intersection_update(self.c_map[InputLetter])
            NewLabel[2*NewNV-1].append( ({InputLetter},self.c_map[InputLetter]) )
            
        #We get a  bit flip in this case if NV < 3. 
            if NewNV < 3:
                NewLabel[11] = False
                PresentLetterList = NewLabel[0].word_as_list + NewLabel[2].word_as_list + NewLabel[4].word_as_list
                PresentLetterSet = set(PresentLetterList)
                #Every previously cancelable letter will become uncancelable. 
                #This will create an uncancelable pair if any of them fail to cancel with one another or with the CancelingLetter.
                #We also check whether there are any duplicates among the PresentLetters.
                if len(PresentLetterList)>len(PresentLetterSet) or ( not self._test_set_commutation(PresentLetterSet)):
                    return(None)
                
                #Add every remaining cancelable letter into the uncancelable set
                NewLabel[8]=NewLabel[8].union(PresentLetterSet)
                for letter in PresentLetterSet:
                    NewLabel[9].intersection_update(self.c_map[letter])
            
                #Delete every previously cancelable letter, which appear only in v_1 and v_2
                for i in range(0,3):
                    NewLabel[2*i] = self.WordGeneratorMachine.word([])
                    NewLabel[2*i+1] = []
                #The 12th entry of NewState.label() will count the number of ending transitions that have elapsed.
                try:
                    NewLabel[12] += 1
                except IndexError:
                    NewLabel.append(1)
                NewState = FSMState(self._hashable_label(NewLabel), is_final = (NewLabel[12] == 2))
                NewTransition = FSMTransition(SourceState, NewState, ('-', InputLetter))
                
                
                return(NewState, NewTransition)
    
            #if the bit has not flipped, there is nothing left to do.
            try:
                NewLabel[12] += 1
            except IndexError:
                NewLabel.append(1)
            NewState = FSMState(self._hashable_label(NewLabel), is_final = (NewLabel[12] == 2))
            NewTransition = FSMTransition(SourceState, NewState, ('-', InputLetter))
            return (NewState, NewTransition)

        #If the InputLetter is canceling, then the increase of NV may have created new uncancelable letters.
        NewUncancelableList = []
        for i in range (0, NewNV-1):
            NewUncancelableList.extend(NewLabel[2*i].word_as_list)
            NewLabel[2*i] = self.WordGeneratorMachine.word([])
            NewLabel[2*i+1] = []

        NewUncancelableSet = set(NewUncancelableList)
        RemainingLetterList = NewLabel[0].word_as_list + NewLabel[2].word_as_list + NewLabel[4].word_as_list
        RemainingLetterSet = set(RemainingLetterList)

        #Check if this creates an uncancelable pair.

        if (len(NewUncancelableList) > len(NewUncancelableSet)) or ( not self._test_set_commutation(NewUncancelableSet)) or ( not self._test_set_pair_commutation (NewUncancelableSet, RemainingLetterSet, True) ):
            return(None)

        #If there is no uncancelable pair, it is also possible to create a bit flip. It is not possible for n_v to be greater than 3, and the InputLetter is a letter of v.
        #Therefore, we only check if the InputLetter commutes with and follows every RemainingLetter.
        
        LettersFollowingAllRemainingLetters = copy.copy(self.alphabet)
        for letter in RemainingLetterSet:
            LettersFollowingAllRemainingLetters.intersection_update(self.greater_star[letter])
        
        if InputLetter in LettersFollowingAllRemainingLetters:
            #Every RemainingLetter will become uncancelable. Check whether this creates an uncancelable pair.
            if (len(RemainingLetterList) > len(RemainingLetterSet) or not self._test_set_commutation(RemainingLetterSet)):
                return(None)
            
            NewLabel[8] = NewLabel[8].union(RemainingLetterSet)
            for letter in RemainingLetterSet:
                    NewLabel[9].intersection_update(self.c_map[letter])
            for i in range(0,4):
                NewLabel[2*i] = self.WordGeneratorMachine.word([])
                NewLabel[2*i+1] = []
                
            #the InputLetter becomes the only cancelable letter if it is in subword 3. Otherwise it becomes uncancelable as well.
            
            if NewNV == 3:
                NewLabel[4] = self.WordGeneratorMachine.word([InputLetter])
                NewLabel[5].append(({InputLetter}, self.c_map[InputLetter]))
                NewLabel[11] = True

                try:
                    NewLabel[12] += 1
                except IndexError:
                    NewLabel.append(1)
                NewState = FSMState(self._hashable_label(NewLabel), is_final = (NewLabel[12] == 2))
                NewTransition = FSMTransition(SourceState, NewState, ('-',InputLetter))
                return (NewState, NewTransition)

            #The InputLetter becomes uncancellable. We know already that it commutes with every other letter.
            NewLabel[8].add(InputLetter)
            NewLabel[9].intersection_update(self.c_map[InputLetter])
            try:
                NewLabel[12] += 1
            except IndexError:
                NewLabel.append(1)
            NewState = FSMState(self._hashable_label(NewLabel), is_final = (NewLabel[12] == 2))
            NewTransition = FSMTransition(SourceState, NewState, ('-',InputLetter))
            return (NewState, NewTransition)

        #If the bit has not flipped, then we process the InputLetter as a canceling letter.
        NewLabel = self.process_canceling_letter(NewLabel, InputLetter, SubwordDict)
        if NewLabel is None:
            return (None)
        try:
            NewLabel[12] += 1
        except IndexError:
            NewLabel.append(1)
        NewState = FSMState(self._hashable_label(NewLabel), is_final = (NewLabel[12] == 2))
        NewTransition = FSMTransition(SourceState, NewState, ('-',InputLetter))
        return (NewState, NewTransition)

    def _mutable_label(self, HashableLabel:tuple) -> list:
        '''
        This method takes a hashable label, as described above, and renders it mutable.
        '''
    
        MutableLabel = list(HashableLabel)
        
        for i in range (0, 4):
            MutableLabel[2*i] = self.WordGeneratorMachine.word(HashableLabel[2*i])
            NewList = []
            for (FirstLetterTuple, PotentialFirstLetterTuple) in HashableLabel[2*i+1]:
                NewList.append( (set(FirstLetterTuple), set(PotentialFirstLetterTuple)) )
            MutableLabel[2*i+1] = NewList
        MutableLabel[8] = set(HashableLabel[8])
        MutableLabel[9] = set(HashableLabel[9])
        
        return(MutableLabel)
    
    def _hashable_label(self, MutableLabel:list) -> tuple:
        '''
        This method takes a mutable state label as described above, and renders it hashable.
        '''
        for i in range (0, 4):
            MutableLabel[2*i] = MutableLabel[2*i].word_as_tuple()
            NewList = []
            #Coerce the first letter data into tuples 
            for (FirstLetterSet, PotentialFirstLetterSet) in MutableLabel[2*i+1]:
                NewList.append( ( tuple(sorted(FirstLetterSet, key = lambda x: self.o_map[x])), tuple(sorted(PotentialFirstLetterSet, key = lambda x: self.o_map[x])) ) )
            MutableLabel[2*i+1]=tuple(NewList)
        if isinstance(MutableLabel[8], set):
            MutableLabel[8] = tuple(sorted(MutableLabel[8], key = lambda x: self.o_map[x]))
        if isinstance(MutableLabel[9], set):
            MutableLabel[9] = tuple(sorted(MutableLabel[9], key = lambda x: self.o_map[x]))
    
        return(tuple(MutableLabel))

    def _test_set_commutation(self, LetterSet: set) -> bool:
        #This method takes as input a set of characters in self.alphabet and checks whether they pairwise commute.
        LetterList = sorted(LetterSet, key = lambda x: self.o_map[x])
        for i in range(0, len(LetterList)):
            #There is no need to include LetterList[i] in the set of letters it commutes with, because LetterSet is a set. We will never test its commutation against itself.
            IthLetterNeighbors = self.c_map[LetterList[i]]
            for j in range(i+1, len(LetterList)):
                if j not in IthLetterNeighbors:
                    return(False)
        return(True)

    def _test_set_pair_commutation(self, FirstLetterSet: set, SecondLetterSet: set, AllDistinct = False) -> bool:
        #This method takes two sets of characters in self.alphabet and checks whether each letter in the first set commutes with each letter in the second set.
        #Optionally, it tests whether the two sets are disjoint. In our use cases, an uncancelable letter appearing in a word must be preceded by another letter that creates an uncancelable pair.
        #Therefore, AllDistinct = True will sometimes serve to prune states that we will not use.

        if AllDistinct and (FirstLetterSet.intersection(SecondLetterSet) != set()):
            return(False)
        for LetterPair in product(FirstLetterSet, SecondLetterSet):
            if not LetterPair[0] in (self.c_map[LetterPair[1]].union({LetterPair[1]})):
                return(False)
        return(True)
       
        

    


In [26]:
class RipsHorosphereGenerator:
    def __init__(self, commutation_dict: dict[str, set], order_dict: dict[str, int], ray: list[str]):
        self.c_map = commutation_dict
        self.o_map = order_dict
        self.alphabet = set().union(letter for letter in self.o_map)
        self.ray = ray
        fsm_gen = Rips_FSM_Generator(self.c_map, self.o_map, self.ray)
        self.suffix_generator = fsm_gen.shortlex_suffix_machine
        self.geodesic_suffix_machine = fsm_gen.geodesic_suffix_machine
        self.ray_excluder = fsm_gen.__FSM_Generator_first_letter_excluder(set(ray))
        self.word_gen = WordGenerator(self.c_map, self.o_map)

    def get_all_length_n_suffixes(self, n:int) -> list:
        """
        Finds all suffixes of length at most n by a BFS.

        :param n: The depth with which the BFS will be conducted.
        :return: A list of all shortlex suffixes of length at most n, formatted as instances of the Word class.
        """
        
        n += 1
        origin = self.suffix_generator.initial_states()[0]

        # All elements of the frontier are of the form (FSMState, current depth, word)
        frontier = [(origin, 0, self.word_gen.word([]))]
        # We will add the words to words_out
        words_out = []

        while frontier:
            state, depth, word = frontier.pop(0)
            # Only go to depth n
            if depth == n:
                continue

            words_out.append(word)

            # Continue traversing
            for transition in self.suffix_generator.iter_transitions(state):
                frontier.append(transition.to_state, depth+1, self.word_gen.word(word.word_as_list + next_letter))

        return words_out

    def calculate_same_length_rips_adjacencies(self, suffix:Word) -> list:
        """
        Given a shortlex suffix, find all suffixes of the same length on the same horosphere that are at distance 2.

        :param suffix: A shortlex suffix.
        :return: The list of all suffixes of the same length as suffix that are at distance 2 from suffix.
        """

        SuffixMachineState = self.suffix_generator.process(suffix.word_as_list)
        
        if not SuffixMachineState[0]:
            raise ValueError('The input is not a shortlex suffix')
        
        # Edge case: empty string "". There are no other length-0 words 
        if len(suffix) == 0:
            return []

        GeodesicSuffixMachineState = self.geodesic_suffix_machine.process(suffix.word_as_list)[1]
        adjacencies = []
        
        for last_letter in GeodesicSuffixMachineState.label()[0]:

            # Remove the last instance of last_letter from the word.
            reversed_word = suffix.word_as_list[::-1]
            deleted_word = (reversed_word[0:reversed_word.index(last_letter):]+reversed_word[reversed_word.index(last_letter)+1::])[::-1]

            deleted_word_state = self.geodesic_suffix_machine.process(deleted_word.word_as_list)[1]

            #The valid letters are those that will not cancel, will not join the prefix, and are not last_letter itself
            connecting_letters = self.alphabet.difference(set(deleted_word_state.label()[0]).union(set(deleted_word_state.label()[1])).union({last_letter}))

            # Lengthen the reduced word by its geodesic legal next letters to get all same length adjacent suffixes
            for letter in connecting_letters:
                adjacencies.append(deleted_word.copy().shortlex_append(letter))

        return adjacencies

    def calculate_different_length_rips_adjacencies(self, suffix: Word, mode:int) -> list:
        """
        Given a suffix, this method finds all suffixes that are 1 letter shorter so that the related words are at distance 2.

        :param suffix: A shortlex suffix.
        :param mode: either 0 or 1. The prefix letter to be deleted or added is self.ray[mode].
        :return: The list of all such suffixes.
        """

        suffix_state = self.suffix_generator.process(suffix.word_as_list)[1]

        adjacencies = []

        #Edge case: suffixes of length 0.
        #There are no suffixes of negative length.
        if len(suffix) == 0:
            return adjacencies

        GeodesicSuffixMachineState = self.geodesic_suffix_machine.process(suffix.word_as_list)[1]

        for last_letter in GeodesicSuffixMachineState.label()[0]:

            # Remove the last instance of last_letter from the word.
            reversed_word = suffix.word_as_list[::-1]
            deleted_word = (reversed_word[0:reversed_word.index(last_letter):]+reversed_word[reversed_word.index(last_letter)+1::])[::-1]

            #Check whether to keep this word.
            if self.ray[mode] in set(self.ray_excluder.process(deleted_word.word_as_list)[1].label()):
                adjacencies.append(deleted_word)
                
        return adjacencies

    def calculate_rips_adjacencies(self, suffix:Word, BusemannValue: int) -> list:
        """
        Use "calculate_same_length_rips_adjacencies" and "calculate_different_length_rips_adj" to find all adjacent words which are at most as long as suffix.

        :param suffix: A shortlex suffix.
        :return: All adjacent suffixes at most as long as "suffix".
        """

        adjacencies = []

        mode = 1-((BusemannValue - len(suffix))%2)
        
        adjacencies.extend(self.calculate_same_length_rips_adjacencies(suffix))
        adjacencies.extend(self.calculate_different_length_rips_adjacencies(suffix, mode))
        return adjacencies

    def calculate_horosphere_edges(self, suffix_list: list, BusemannValue: int) -> list:
        """
        Calculate the set of edges from entries of suffix_list to shorter suffixes.

        :param suffix_list: A list of suffixes.
        :param BusemannValue: Value of the Busemann function for the horosphere.
        :return: A list of all pairs of suffixes whose corresponding words are at distance 2, 
        such that the first is an entry of suffix_list and the second is no longer than the first.
        """

        edges = []

        for suffix in suffix_list:
            edges.append( (str(suffix), str(newsuffix)) for newsuffix in self.calculate_rips_adjacencies(suffix, BusemannValue) )
        return(edges)
        
    def horosphere_as_networkx(self, length: int, BusemannValue: int):
        suffixes = self.get_all_length_n_suffixes(length)
        print(f"Suffixes of length {length} calculated: \n\t\t {len(words)} words found")
        edges = self.calculate_horosphere_edges(suffixes, BusemannValue)
        print(f"Words processing completed: \n\t\t {len(words)} words processed")

        G = networkx.Graph()
        G.add_edges_from(processed_edges)
        print(f"Length {length} Horosphere generated: \n\t\t {G}")
        return G

In [36]:
class DivergenceHorosphereGenerator:
    def __init__(self, commutation_dict: dict[str, set], order_dict: dict[str, int], ray: list[str]):
        self.c_map = commutation_dict
        self.o_map = order_dict
        self.alphabet = set().union(letter for letter in self.o_map)
        self.ray = ray
        fsm_gen = Divergence_FSM_Generator(self.c_map, self.o_map, self.ray)
        self.horocyclic_suffix_machine_1234 = fsm_gen.horocyclic_suffix_machine_1234()
        self.horocyclic_suffix_machine_1256 = fsm_gen.horocyclic_suffix_machine_1256()
        self.shortlex_machine = fsm_gen._Rips_FSM_Generator__shortlex_machine()
        self.geodesic_machine = fsm_gen._Rips_FSM_Generator__geodesic_machine()
        self.geodesic_suffix_machine = fsm_gen.geodesic_suffix_machine()
        self.word_gen = WordGenerator(self.c_map, self.o_map)

        #These dictionaries will keep track of which letters appear in which subwords of horocyclic suffixes. 
        SubwordDict1234 = {}
        SubwordDict1256 = {}
        SubwordDictDifferentLength = {}
        #The letters that commute with and precede both ray letters appear first in w_1
        for letter in self.c_map[self.ray[0]].intersection(fsm_gen.lesser_star[self.ray[1]]):
            SubwordDict1234[letter] = 1
            SubwordDict1256[letter] = 1
            SubwordDictDifferentLength[letter] = 1
        #The letters that commute with both ray letters and precede only the first appear first in w_2
        for letter in fsm_gen.lesser_star[self.ray[0]].intersection(fsm_gen.greater_star[self.ray[1]]):
            SubwordDict1234[letter] = 2
            SubwordDict1256[letter] = 2
            SubwordDictDifferentLength[letter] = 2
        #For words of the form w_1w_2w_3w_4, the letters that commute with and precede the second ray letter, but do not commute with the first, appear in w_3.
        for letter in fsm_gen.lesser_star[self.ray[1]].difference(self.c_map[self.ray[0]]):
            SubwordDict1234[letter] = 3
        #All the rest appear only in w_4
        for letter in fsm_gen.alphabet.difference(fsm_gen.lesser_star[self.ray[1]].union(fsm_gen.lesser_star[self.ray[0]].intersection(self.c_map[self.ray[1]]))):
            SubwordDict1234[letter] = 4
        #For words of the form w_1w_2w_5w_6, the letters that commute with and precede the first ray letter, but do not commute with the second, appear in w_5
        for letter in fsm_gen.lesser_star[self.ray[0]].difference(self.c_map[self.ray[1]]):
            SubwordDict1256[letter] = 3
        #All the rest appear only in w_6
        for letter in fsm_gen.alphabet.difference(fsm_gen.lesser_star[self.ray[0]].union(fsm_gen.lesser_star[self.ray[1]].intersection(self.c_map[self.ray[0]]))):
            SubwordDict1256[letter] = 4
        #For pairs of words of different length, every other letter will be in subword 3.
        for letter in self.alphabet:
            try:
                SubwordDictDifferentLength[letter]
            except KeyError:
                SubwordDictDifferentLength[letter] = 3

        self.same_length_edge_checker1234 = fsm_gen.horocyclic_edge_checker(SubwordDict1234)
        self.same_length_edge_checker1256 = fsm_gen.horocyclic_edge_checker(SubwordDict1256)
        self.different_length_edge_checker = fsm_gen.horocyclic_edge_checker(SubwordDictDifferentLength)
        
        self.clique_dimension = self._determine_clique_dimension_recursive(0, self.alphabet)

    def _determine_clique_dimension_recursive(self, running_total: int, allowable_set:set) -> int:
        '''
        A recursive method to determine the clique dimension of the defining graph.

        :param running_total: the number of letters already in the clique.
        :param allowable_set: the collection of letters commuting with those already in the clique.
        :return: the maximum size of a clique containing the letters already included.
        '''

        if allowable_set == set():
            return running_total

        n = running_total
        
        for letter in allowable_set:
            n = max(n, self._determine_clique_dimension_recursive(running_total+1, allowable_set.intersection(self.c_map[letter])))
        return(n)

    def get_all_length_n_horocyclic_suffixes(self, n:int, mode:bool) -> list:
        '''
        Generate the list of horocyclic suffixes of length n or less.

        :param mode: False if the value of the Busemann Function is even, True if odd.

        :return: A list of all horocyclic suffixes for the desired value of the Busemann Function of up to length n.
        '''
        
        HorocyclicSuffixList = []
        #EvenList = []
        #OddList = []
        Frontier = []

        if mode:
            EvenLengthGenerator = self.horocyclic_suffix_machine_1234
            OddLengthGenerator = self.horocyclic_suffix_machine_1256
        else:
            EvenLengthGenerator = self.horocyclic_suffix_machine_1256
            OddLengthGenerator = self.horocyclic_suffix_machine_1234
        
        #EvenList.append(self.word_gen.horocyclic_word([[], [], [], []]))
        NewWord = self.word_gen.horocyclic_word([[], [], [], []], mode)
        HorocyclicSuffixList.append(NewWord)
        Frontier.append((EvenLengthGenerator.initial_states()[0], 0, NewWord))
        
        #Edge case: n==0

        if n == 0:
            return HorocyclicSuffixList

        for transition in OddLengthGenerator.transitions(OddLengthGenerator.initial_states()[0]):
            NextStateLabel = transition.to_state.label()
            if NextStateLabel[0]:
                #If NextStateLabel[0] is 1, that means that the letter is not in w_1 or w_2. 
                #In particular, the label is of the form (1, (( int, SubwordState), (LetterExcluderState) ) ) where int is 0 for the third subword or 1 for the fourth.
                SubwordList = [[], [], [], []]
                SubwordList[2+NextStateLabel[1][0].label()[0]] = [transition.word_in]
            else:
                #If NextStateLabel[0] is 0, that means that the letter is in w_1 or w_2.
                #In particular, the label is of the form (0, (int, SubwordState)) where int is 0 for the first subword or 1 for the second.
                SubwordList = [[], [], [], []]
                SubwordList[NextStateLabel[1][0]] = [transition.word_in]
            NewWord = self.word_gen.horocyclic_word(SubwordList, not mode)
            HorocyclicSuffixList.append(NewWord)
            #OddList.append(ResultingWord)
            Frontier.append((transition.to_state, 1, NewWord))

        #We will lengthen words 2 letters at a time.
        while Frontier:
            state, depth, word = Frontier.pop(0)
            if depth > n-2:
                continue
            if depth%2:
                for FirstTransition in OddLengthGenerator.transitions(state):
                    for SecondTransition in OddLengthGenerator.transitions(FirstTransition.to_state):
                        NewWord = word.copy()

                        #Append the first letter to the relevant subword
                        if FirstTransition.to_state.label()[0]:
                            NewWord.append(FirstTransition.word_in, 2+FirstTransition.to_state.label()[1][0].label()[0])
                        else:
                            NewWord.append(FirstTransition.word_in, FirstTransition.to_state.label()[1][0])
                            
                        #Append the second letter to the relevant subword.
                        if SecondTransition.to_state.label()[0]:
                            NewWord.append(SecondTransition.word_in, 2+SecondTransition.to_state.label()[1][0].label()[0])
                        else:
                            NewWord.append(SecondTransition.word_in, SecondTransition.to_state.label()[1][0])
                            
                        HorocyclicSuffixList.append(NewWord)
                        Frontier.append((SecondTransition.to_state, depth+2, NewWord))
                        #OddList.append(NewWord)
            else:
                for FirstTransition in EvenLengthGenerator.transitions(state):
                    for SecondTransition in EvenLengthGenerator.transitions(FirstTransition.to_state):
                        NewWord = word.copy()

                        #Append the first letter to the relevant subword
                        if FirstTransition.to_state.label()[0]:
                            NewWord.append(FirstTransition.word_in, 2+FirstTransition.to_state.label()[1][0].label()[0])
                        else:
                            NewWord.append(FirstTransition.word_in, FirstTransition.to_state.label()[1][0])
                            
                        #Append the second letter to the relevant subword.
                        if SecondTransition.to_state.label()[0]:
                            NewWord.append(SecondTransition.word_in, 2+SecondTransition.to_state.label()[1][0].label()[0])
                        else:
                            NewWord.append(SecondTransition.word_in, SecondTransition.to_state.label()[1][0])
                            
                        HorocyclicSuffixList.append(NewWord)
                        Frontier.append((SecondTransition.to_state, depth+2, NewWord))
                        #EvenList.append(NewWord)

        return HorocyclicSuffixList
            
    def calculate_same_length_divergence_adjacencies(self, HorocyclicSuffix:HorocyclicWord) ->list:
        '''
        Given a horocyclic suffix, find all horocyclic suffixes of the same length on the same horosphere such that there is an edge between the two in the divergence graph. 
        
        :param HorocyclicSuffix: A horocyclic suffix.
        :return: The list of all horocyclic suffixes of the same length as HorocyclicSuffix such that the two have close successors.
        '''

        BacktrackedWords = []
        CandidateList = []
        
        #Forbidding loops in the graph.
        FinishedWords = [HorocyclicSuffix]
        
        Adjacencies = []

        #We construct the list of horocyclic suffixes of the same length and at distance at most self.clique_dimension away.
        #We begin by deleting letters from HorocyclicSuffix

        #Edge case: HorocyclicSuffix is short enough that it backtracks all the way to the identity.
        if len(HorocyclicSuffix) <self.clique_dimension:
            BacktrackedWords = [self.word_gen.horocyclic_word([[],[],[],[]], HorocyclicSuffix.mode)]
        else:
            BacktrackedWords = self._backtracking_recursive(HorocyclicSuffix, self.clique_dimension, self.shortlex_machine.initial_states()[0])

        #We can follow this word by any geodesic word, as long as the result is still a suffix. This will be be potentially highly non-unique, but it is not obvious how to avoid repetition
        for word in BacktrackedWords:
            CandidateList.extend(self._geodesic_successor_horocyclic_suffixes(word, min(len(HorocyclicSuffix),self.clique_dimension), self.geodesic_suffix_machine.process(word.word_as_list)[1]))

        while len(CandidateList) > 0:
            CurrentCandidate = CandidateList.pop(0)
            if CurrentCandidate in FinishedWords:
                continue

            #The edge checker machine wants an input tape that consists of pairs of characters.
            InputList = list(zip(HorocyclicSuffix.word_as_list+['-'], CurrentCandidate.word_as_list+['-']))
            
            
            if HorocyclicSuffix.mode:
                InputAccepted, EndState = self.same_length_edge_checker1234.process(InputList)[:2]
            else:
                InputAccepted, EndState = self.same_length_edge_checker1256.process(InputList)[:2]
            if not InputAccepted:
                FinishedWords.append(CurrentCandidate)
                continue

            #The same length edge checker only tells us that no non-cancelable pair has been created.
            #To check whether the two have close successors, we need to check whether there is an infinite alternation of some pair of letters as described in Proposition 5.2.10

            CancelingWord = EndState.label()[6]
            #If the cancelable letters were part of the second word (CurrentCandidate) then it is the first word (HorocyclicSuffix) that needs to be elongated to achieve full cancelation
            if EndState.label()[11]:
                ElongatedWord = HorocyclicSuffix.word_as_list + list(CancelingWord)
                NonElongatedWord = CurrentCandidate.word_as_list
            #Otherwise, it is CurrentCandidate that needs to be elongated.
            else:
                ElongatedWord = CurrentCandidate.word_as_list + list(CancelingWord)
                NonElongatedWord = HorocyclicSuffix.word_as_list
                
            if HorocyclicSuffix.mode:
                relevant_suffix_machine = self.horocyclic_suffix_machine_1234
            else:
                relevant_suffix_machine = self.horocyclic_suffix_machine_1256
            ElongatedState = relevant_suffix_machine.process(ElongatedWord)[1]
            NonElongatedState = relevant_suffix_machine.process(NonElongatedWord)[1]
            
            LettersCommutingWithClique = set(EndState.label()[9])
            #Check whether any of these letters is permitted by both the above states.

            print('Processing pair ', HorocyclicSuffix.word_as_list, CurrentCandidate.word_as_list)
            print('The elongated word is ', ElongatedWord, ' and the non-elongated word is ', NonElongatedWord)
            print('The state of the elongated word is ', ElongatedState.label())
            print('Its outgoing transitions are ', relevant_suffix_machine.transitions(ElongatedState))
            
            #Recall that transition.word_in outputs a list containing the label of the transition, not the label itself.
            LettersAcceptedFromElongatedState = {str for str in transition.word_in for transition in relevant_suffix_machine.iter_transitions(ElongatedState)}
            #ElongatedTransitionDict = relevant_suffix_machine.transition_dict(ElongatedState)
            LettersAcceptedFromNonElongatedState = {str for str in transition.word_in for transition in relevant_suffix_machine.iter_transitions(NonElongatedState)}
            #NonElongatedTransitionDict = relevant_suffix_machine.transition_dict(NonElongatedState)
        
            for letter in LettersCommutingWithClique:
                if letter in LettersAcceptedFromElongatedState.intersection(LettersAcceptedFromNonElongatedState) and LettersCommutingWithClique.difference(self.c_map[letter].union({letter})) != set():
                    Adjacencies.append(CurrentCandidate)
                    break
            FinishedWords.append(CurrentCandidate)
                    
        return Adjacencies

    def calculate_shorter_divergence_adjacencies(self, HorocyclicSuffix:HorocyclicWord) ->list:
        '''
        Given a horocyclic suffix, find all shorter horocyclic on the same horosphere such that there is an edge between the two in the divergence graph. 
        
        :param HorocyclicSuffix: A horocyclic suffix.
        :return: The list of all horocyclic suffixes of shorter than HorocyclicSuffix such that the two have close successors.
        '''

        Adjacencies = []
        
        if HorocyclicSuffix[3] != []:
            return Adjacencies

        for letter in HorocyclicSuffix[4]:
            if not letter in self.c_map[self.ray[1-int(HorocyclicSuffix.mode)]]:
                return Adjacencies
        
        BacktrackedWords = []
        CandidateList = []
        FinishedWords = []

        #This list will keep track of the letters to add to the shorter word
        ExtensionList = []
        for length in range(0, self.clique_dimension):
            if HorocyclicSuffix.mode ^ length%2:
                ExtensionList.append(self.ray[0])
            else:
                ExtensionList.append(self.ray[1])
        
        if HorocyclicSuffix[4] == []:
            #If so, then every letter of HorocyclicSuffix commutes with both self.ray[0] and self.ray[1], so that they all commute with one another.
            #Therefore, these words backtrack all the way to the identity.
            CandidateList = self.get_all_length_n_horocyclic_suffixes(len(HorocyclicSuffix)-1, not (HorocyclicSuffix.mode ^ (len(HorocyclicSuffix)%2)))

        else:
            #In this case, the only shorter words that HorocyclicSuffix can be adjacent to are words that are 1 letter shorter. These must all be of the opposite form to HorocyclicSuffix
            if len(HorocyclicSuffix) < self.clique_dimension:
                BacktrackedWords = [self.word_gen.horocyclic_word([[],[],[],[]], not HorocyclicSuffix.mode)]
            else:
                StartingWord = self._switch_form_special_case(HorocyclicSuffix)
                BacktrackedWords = self._backtracking_recursive(StartingWord, self.clique_dimension, self.shortlex_machine.initial_states()[0])
                                
            for word in BacktrackedWords:
                CandidateList.extend(self._geodesic_successor_horocyclic_suffixes(word, min(len(HorocyclicSuffix),self.clique_dimension)), self.geodesic_suffix_machine.process(word.word_as_list)[1])
            
        while CandidateList:
            CurrentCandidate = CandidateList.pop(0)
            if CurrentCandidate in FinishedWords:
                continue

            CandidateInput = CurrentCandidate[0] + CurrentCandidate[1] + CurrentCandidate[2] + ExtensionList[0:len(HorocyclicSuffix)-len(CurrentCandidate):] + CurrentCandidate[3]
            InputList = list(zip(HorocyclicSuffix.word_as_list+['-'], CandidateInput+['-']))

            EndState = self.different_length_edge_checker.process(InputList)
            if not EndState[0]:
                FinishedWords.append(CurrentCandidate)
                continue

            #The same length edge checker only tells us that no non-cancelable pair has been created.
            #To check whether the two have close successors, we need to check whether there is an infinite alternation of some pair of letters as described in Proposition 5.2.10

            CancelingWord = EndState.label()[4]
             
            if HorocyclicSuffix.mode:
                if EndState.label()[11]:
                    HorocyclicState = self.horocyclic_suffix_machine_1234.process(HorocyclicSuffix.word_as_list + list(CancelingWord))[1]
                    HorocyclicTransitionDict = self.horocyclic_suffix_machine_1234.transition_dict(HorocyclicState)
                else:
                    HorocyclicState = self.horocyclic_suffix_machine_1234.process(HorocyclicSuffix.word_as_list)[1]
                    HorocyclicTransitionDict = self.horocyclic_suffix_machine_1234.transition_dict(HorocyclicState)
            else:
                if EndState.label()[11]:
                    HorocyclicState = self.horocyclic_suffix_machine_1256.process(HorocyclicSuffix.word_as_list + list(CancelingWord))[1]
                    HorocyclicTransitionDict = self.horocyclic_suffix_machine_1256.transition_dict(HorocyclicState)
                else:
                    HorocyclicState = self.horocyclic_suffix_machine_1256.process(HorocyclicSuffix.word_as_list)[1]
                    HorocyclicTransitionDict = self.horocyclic_suffix_machine_1256.transition_dict(HorocyclicState)

            if CurrentCandidate.mode:
                if EndState.label()[11]:
                    CandidateState = self.horocyclic_suffix_machine_1234.process(CurrentCandidate.word_as_list)[1]
                    CandidateTransitionDict = self.horocyclic_suffix_machine_1234.transition_dict(CandidateState)
                else:
                    CandidateState = self.horocyclic_suffix_machine_1234.process(CurrentCandidate.word_as_list+ list(CancelingWord))[1]
                    CandidateTransitionDict = self.horocyclic_suffix_machine_1234.transition_dict(CandidateState)
            else:
                if EndState.label()[11]:
                    CandidateState = self.horocyclic_suffix_machine_1256.process(CurrentCandidate.word_as_list)[1]
                    CandidateTransitionDict = self.horocyclic_suffix_machine_1256.transition_dict(CandidateState)
                else:
                    CandidateState = self.horocyclic_suffix_machine_1256.process(CurrentCandidate.word_as_list+ list(CancelingWord))[1]
                    CandidateTransitionDict = self.horocyclic_suffix_machine_1256.transition_dict(CandidateState)

            LettersCommutingWithClique = set(EndState.label()[9])
            #Check whether any of these letters is permitted by both the above states.
        
            for letter in LettersCommutingWithClique:
                #Check if this letter can be written after both words
                try:
                    HorocyclicTransitionDict[letter]
                    CandidateTransitionDict[letter]
                except KeyError:
                    continue
                #Check if there is another letter in the alphabet which does not commute with the given letter
                if LettersCommutingWithClique.difference(self.c_map[letter].union({letter})) != set():
                    Adjacencies.append(CurrentCandidate)
                    break
            FinishedWords.append(CurrentCandidate)
                    
        return Adjacencies
        
    def calculate_divergence_adjacencies(self, HorocyclicSuffix:HorocyclicWord) -> list:
        """
        Use "calculate_same_length_divergence_adjacencies" and "calculate_shorter_divergence_adjacencies" to find all adjacent words which are at most as long as suffix.

        :param HorocyclicSuffix: A horocyclic suffix.
        :return: All adjacent suffixes at most as long as "suffix".
        """

        adjacencies = []
        
        adjacencies.extend(self.calculate_same_length_divergence_adjacencies(HorocyclicSuffix))
        adjacencies.extend(self.calculate_different_length_rips_adjacencies(HorocyclicSuffix))
        
        return adjacencies

    def calculate_divergence_horosphere_edges(self, HorocyclicSuffix_list: list) -> list:
        """
        Calculate the set of edges from entries of suffix_list to shorter suffixes.

        :param HorocyclicSuffix_list: A list of horocyclic suffixes.
        :return: A list of all pairs of horocyclic suffixes whose corresponding words have close successors for all time 
        such that the first is an entry of HorocyclicSuffix_list and the second is no longer than the first.
        """

        edges = []

        for suffix in HorocyclicSuffix_list:
            edges.append( (str(suffix), str(newsuffix)) for newsuffix in self.calculate_divergence_adjacencies(suffix) )
        return(edges)
        
    def horosphere_as_networkx(self, length: int, BusemannValue: int):
        suffixes = self.get_all_length_n_horocyclic_suffixes(length, bool(BusemannValue % 2))
        print(f"Suffixes of length {length} calculated: \n\t\t {len(suffixes)} words found")
        edges = self.calculate_divergence_horosphere_edges(suffixes)
        print(f"Words processing completed: \n\t\t {len(suffixes)} words processed")

        G = networkx.Graph()
        G.add_edges_from(processed_edges)
        print(f"Length {length} Horosphere generated: \n\t\t {G}")
        return G

    def _backtracking_recursive(self, word:HorocyclicWord, StepCount:int, ShortlexState:FSMState) -> list:
        '''
        Recursively find all horocyclic suffixes word' so that, as geodesic words word=word' + tail, where tail is a shortlex word StepCount letters long.
        The assumption that tail is shortlex is only relevant to make the outputs unique.

        :param word: the word that has been reached so far.
        :param StepCount: the number of backtracking steps that still remain.
        :param ShortlexState: the current state in the shortlex machine 
        :return: a list without repetition of the HorocyclicWords word' as above.
        '''

        if StepCount == 0:
            return([word])

        ResultingWords = []
        #We backtrack by the letters that are both final letters of word and still allowed by the shortlex state.
        LastLetters = set(self.geodesic_machine.process(word.word_as_list)[1].label())
        for transition in self.shortlex_machine.transitions(ShortlexState):
            if transition.word_in in LastLetters:
                for i in reversed(range(0,4)):
                    if transition.word_in in set(word[i]):
                        NewSubwordList = word.SubwordList
                        ReversedSubword = word.SubwordList[i][::-1]
                        DeletedSubword = (ReversedSubword[0:ReversedSubword.index(transition.word_in):]+ReversedSubword[ReversedSubword.index(transition.word_in)+1::])[::-1]
                        NewSubwordList[i] = DeletedSubword
                        NewWord = self.word_gen.horocyclic_word(NewSubwordList, word.mode)
                        break
                ResultingWords.extend(self._backtracking_recursive(NewWord, StepCount-1, transition.to_state))
        return ResultingWords

    def _geodesic_successor_horocyclic_suffixes(self, word:HorocyclicWord, StepCount:int, GeodesicSuffixState:FSMState) -> list:
        '''
        Recursively find all horocyclic suffixes word' that are StepCount letters longer than word, such that word' is a geodesic successor to word.

        :param word: the successor that has been reached so far.
        :param StepCount: the number of letters still to write.
        :param GeodesicSuffixState: the state of word in the geodesic suffix machine.
        :return: a list with repetition of all the HorocyclicWords word' as above.
        '''
        
        if StepCount == 0:
            return [word]

        ResultingWords = []  
        
        for transition in self.geodesic_suffix_machine.transitions(GeodesicSuffixState):
            #determine which subword transition.word_in should be inserted into.
            if word.mode:
                EarliestPossibleSubword = SubwordDictionary1234[transition.word_in]
            else:
                EarliestPossibleSubword = SubwordDictionary1256[transition.word_in]
            
            for i in reversed(range(EarliestPossibleSubword-1,4)):
                #transition.word_in belongs in subword i either if it cannot commute to an earlier subword, or if subword i is the earliest subword the letter is allowed in.
                if i == EarliestPossibleSubword-1 or (not fsm_gen.test_set_pair_commutation({transition.word_in},set(word[i]))):
                    NewSubwordList = word.SubwordList
                    NewSubword = self.word_gen.word(word[i])
                    NewSubword.shortlex_append(transition.word_in)
                    NewSubwordList[i] = NewSubword.word_as_list
                    NewWord = self.word_gen.horocyclic_word(NewSubwordList, word.mode)
                    break
            ResultingWords.extend(self._geodesic_successor_horocyclic_suffixes(NewWord, StepCount-1, transition.to_state))
            
        return ResultingWords

    def _switch_form_special_case(self, word:HorocyclicWord) -> HorocyclicWord:
        '''
        Given a horocyclic suffix of form 1234, return an equal horocyclic suffix of form 1256, or vice versa.
        Only implemented in the case that subword 3 is empty
        and that subword 4 consists of letters commuting with the ray letter that is between word_5 and word_6 if word is of form 1234, or vice versa.
    
        With this assumption, once a letter of word[4] is read that follows this ray letter, no letter preceding the ray letter can be written until a letter is written that does not commute with it
        Therefore, the first time a letter of word[4] follows the relevant ray letter is the break between the resulting subwords.
        This takes linear time to computer. Without the assumption, it would take quadratic time.
    
        :param word: A HorocyclicWord with empty third subword and subword 4 satisfying the condition above.
        :return: An equivalent HorocyclicWord of the opposite form to that of word.
        
        '''
    
        if word[3] != []:
            raise ValueError('Not implemented in the general case')
    
        #We will need to insert a letter into the fourth subword of word.
        #If word is of the form 1234, then the letter to be inserted is self.ray[0], and vice versa.
        SplitLetter = self.ray[1-int(word.mode)]
        
        #sentinel value
        SplitIndex = None
        
        for index in  range(0, len(word[4])):
            if not word[4][index] in self.c_map[SplitLetter]:
                raise ValueError('Not implemented in the general case')
            if self.o_map[word[4][index]] > self.o_map[SplitLetter] and SplitIndex is None:
                SplitIndex = index
                
        #At the end of either loop, SplitIndex records either the first instance of a letter commuting with and following SplitLetter
        #Or, if each letter commutes with and precedes SplitLetter, then SplitIndex still has the sentinel value None.
        if SplitIndex is None:
            NewWord = self.word_gen.horocyclic_word([word[1], word[2], word[4], []], not word.mode)
        else:
            NewWord = self.word_gen.horocyclic_word([word[1], word[2], word[4][:SplitIndex:], word[4][SplitIndex::]], not word.mode)
    
        return NewWord
            
            
        
        



In [37]:

#Probably some kind of virtual theta graph? This one is guaranteed to have some non-terminal states, which is its point for debugging
'''

my_c_map = {
    'a': {'b', 'g', 'h'}, 'b': {'a', 'z'}, 'c': {'d', 'e', 'g', 'z'}, 'd': {'c', 'e', 'f', 'z'},
    'e': {'c', 'd', 'f', 'z'}, 'f': {'d', 'e', 'h', 'z'}, 'g': {'a', 'c'}, 'h': {'a', 'f'}, 'z': {'b', 'c', 'd', 'e', 'f'} 
}

my_o_map = {
    'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'z': 8
}

my_ray = ['a', 'z']



#For the Sierpinski Carpet case:
my_c_map = {
    'a' : {'b', 'e', 'f', 'j'}, 'b' : {'a', 'c', 'f', 'g'},
    'c' : {'b', 'd', 'g', 'h'}, 'd' : {'c', 'e', 'h', 'i'},
    'e' : {'a', 'd', 'i', 'j'}, 'f' : {'a', 'b', 'g', 'j'},
    'g' : {'b', 'c', 'f', 'h'}, 'h' : {'c', 'd', 'g', 'i'},
    'i' : {'d', 'e', 'h', 'j'}, 'j' : {'a', 'e', 'f', 'i'}
}

my_o_map = {'a' : 0, 'b' : 1, 'c' : 2, 'd' : 3, 'e' : 4, 'f' : 5, 'g' : 6, 'h' : 7, 'i' : 8, 'j' : 9}
my_ray = ['a', 'g']

'''

#For the virtual surface group case:

my_c_map = {'a': {'b', 'e'}, 'b': {'a', 'c'}, 'c': {'b', 'd'}, 'd': {'e', 'c'}, 'e': {'a', 'd'}}
my_o_map = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}
my_ray = ['a', 'c']



In [38]:
Object = DivergenceHorosphereGenerator(my_c_map, my_o_map, my_ray)
FSMGenerator = Divergence_FSM_Generator(my_c_map, my_o_map, my_ray)

You have requested the shortlex machine on an empty alphabet. Returning an automaton that accepts only the empty string
You have requested the shortlex machine on an empty alphabet. Returning an automaton that accepts only the empty string
You have requested the shortlex machine on an empty alphabet. Returning an automaton that accepts only the empty string
Completed the uncancelable states. There are  2  of them.
Completed the first batch of transitions. There are  30 transitions and  16  states in total
Completed the nonterminal transitions. There are  70  transitions and  22  states in total
completed the final transitions. There are  92 transitions and  17  final states in total
Completed the uncancelable states. There are  2  of them.
Completed the first batch of transitions. There are  30 transitions and  16  states in total
Completed the nonterminal transitions. There are  70  transitions and  22  states in total
completed the final transitions. There are  92 transitions and  17

In [16]:
Object.horosphere_as_networkx(3, 0)

Suffixes of length 3 calculated: 
		 29 words found
Processing pair  [] []
The elongated word is  []  and the non-elongated word is  []
The state of the elongated word is  (0, (0, ()))
Its outgoing transitions are  [Transition from (0, (0, ())) to (0, (0, ('b',))): 'b'|-, Transition from (0, (0, ())) to (1, ((1, (('a', 'd', 'e'), ('a',))), ())): 'e'|-, Transition from (0, (0, ())) to (1, ((1, (('c', 'd'), ('c',))), ('c',))): 'd'|-]


TypeError: unhashable type: 'list'

In [ ]:
#todo: Make sure that the part where I figure out which letters go in which subwords is correct. 
#Can there be letters which first appear in w_1 or w_2, but which cannot appear in w_3 (or w_4)?

In [35]:
OddLengthGenerator = FSMGenerator.horocyclic_suffix_machine_1234()
OddLengthGenerator.transitions(OddLengthGenerator.initial_states()[0])[1].word_in

You have requested the shortlex machine on an empty alphabet. Returning an automaton that accepts only the empty string


['e']

int